In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import pickle

import neurokit2 as nk

import seaborn as sns

%matplotlib qt
mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('ggplot')
plt.style.use('seaborn-v0_8-whitegrid')

# sns.set_context("talk")
sns.set_palette("Set1")

In [2]:
import pyreadr
from collections import Counter
from functions.bursts import characterize_bursts
from sleep_diary import diary_SPT

# Supplementary - raw acc SMV baseline change

In [112]:
subjects = ["158", "098"]

diary_SPT = {    
    "158": [pd.Timestamp('2024-02-28 23:00:00'), pd.Timestamp('2024-02-29 07:15:00')], # 158 OK
    "098": [pd.Timestamp('2024-03-16 02:01:00'), pd.Timestamp('2024-03-16 09:50:00')], # 098 OK
}

TH_WRIST = 20
TH_ANKLE = 15
TH_TRUNK = 15

for i, sub in enumerate(subjects):

    print(sub)

    start_sleep, end_sleep = diary_SPT[sub]
    lw_df = pd.read_pickle(f"/Volumes/Untitled/rehab/data/{sub}/acc_night/lw.pkl") * 1000
    lw_df_bp = pd.Series(nk.signal_filter(lw_df.values, sampling_rate = 100, lowcut=0.1, highcut=10, method='butterworth', order=8), index = lw_df.index)
    break

158


In [115]:
lmin, lmax = hl_envelopes_idx(lw_df.values, dmin=5, dmax=5)
upper_envelope = lw_df.values[lmax]
lower_envelope = lw_df.values[lmin]
env_diff = pd.Series(lw_df.values[lmax] - lw_df.values[lmin], index = lw_df.index[lmax])
upper_envelope_res = np.interp(np.arange(len(lw_df)), lmax, upper_envelope)
lower_envelope_res = np.interp(np.arange(len(lw_df)), lmin, lower_envelope)
env_diff_res = pd.Series(upper_envelope_res - lower_envelope_res, index = lw_df.index)

In [117]:
plt.figure(figsize=(8, 6))
plt.subplot(2, 1, 1)
plt.plot(lw_df, 'k')
plt.plot(lw_df.index[lmin], lw_df.values[lmin], color = 'r', linewidth = 1.6)
plt.plot(lw_df.index[lmax], lw_df.values[lmax], color = 'g', linewidth = 1.6)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.ylabel("ACC (mg)", fontsize = 16)
plt.legend(["Accelerometer SMV", "Low envelope", "High envelope"], loc = "upper left", frameon = True, fancybox = True, shadow = True, fontsize = 16)
plt.subplot(2, 1, 2, sharex = plt.subplot(2, 1, 1))
plt.plot(env_diff_res, 'b', linewidth = 1.9)

# Methods - Envelope

In [111]:
def hl_envelopes_idx(s, dmin=1, dmax=1, split=False, plot = True):
    """
    Compute high and low envelopes of a signal s
    Parameters
    ----------
    s: 1d-array, data signal from which to extract high and low envelopes
    dmin, dmax: int, optional, size of chunks, use this if the size of the input signal is too big
    split: bool, optional, if True, split the signal in half along its mean, might help to generate the envelope in some cases

    Returns
    -------
    lmin,lmax : high/low envelope idx of input signal s
    """

    # locals min      
    lmin = (np.diff(np.sign(np.diff(s))) > 0).nonzero()[0] + 1 
    # locals max
    lmax = (np.diff(np.sign(np.diff(s))) < 0).nonzero()[0] + 1 
    
    if split:
        # s_mid is zero if s centered around x-axis or more generally mean of signal
        s_mid = np.mean(s) 
        # pre-sorting of locals min based on relative position with respect to s_mid 
        lmin = lmin[s[lmin]<s_mid]
        # pre-sorting of local max based on relative position with respect to s_mid 
        lmax = lmax[s[lmax]>s_mid]

    # global min of dmin-chunks of locals min 
    lmin = lmin[[i+np.argmin(s[lmin[i:i+dmin]]) for i in range(0,len(lmin),dmin)]]
    # global max of dmax-chunks of locals max 
    lmax = lmax[[i+np.argmax(s[lmax[i:i+dmax]]) for i in range(0,len(lmax),dmax)]]

    return lmin, lmax

In [ ]:
subjects = ["158", "098"]

diary_SPT = {    
    "158": [pd.Timestamp('2024-02-28 23:00:00'), pd.Timestamp('2024-02-29 07:15:00')], # 158 OK
    "098": [pd.Timestamp('2024-03-16 02:01:00'), pd.Timestamp('2024-03-16 09:50:00')], # 098 OK
}

TH_WRIST = 20
TH_ANKLE = 15
TH_TRUNK = 15

for i, sub in enumerate(subjects):

    print(sub)

    start_sleep, end_sleep = diary_SPT[sub]
    lw_df = pd.read_pickle(f"/Volumes/Untitled/rehab/data/{sub}/acc_night/lw.pkl") * 1000
    lw_df_bp = pd.Series(nk.signal_filter(lw_df.values, sampling_rate = 100, lowcut=0.1, highcut=10, method='butterworth', order=8), index = lw_df.index)
    break

158


In [6]:
lmin, lmax = hl_envelopes_idx(lw_df_bp.values, dmin=5, dmax=5)
upper_envelope = lw_df_bp.values[lmax]
lower_envelope = lw_df_bp.values[lmin]
env_diff = pd.Series(lw_df_bp.values[lmax] - lw_df_bp.values[lmin], index = lw_df_bp.index[lmax])
upper_envelope_res = np.interp(np.arange(len(lw_df_bp)), lmax, upper_envelope)
lower_envelope_res = np.interp(np.arange(len(lw_df_bp)), lmin, lower_envelope)
env_diff_res = pd.Series(upper_envelope_res - lower_envelope_res, index = lw_df_bp.index)
plt.figure(figsize=(8, 6))
plt.subplot(2, 1, 1)
plt.plot(lw_df_bp, 'k')
plt.plot(lw_df_bp.index[lmin], lw_df_bp.values[lmin], color = 'r', linewidth = 1.6)
plt.plot(lw_df_bp.index[lmax], lw_df_bp.values[lmax], color = 'g', linewidth = 1.6)
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.ylabel("ACC (mg)", fontsize = 16)
plt.legend(["Accelerometer SMV", "Low envelope", "High envelope"], loc = "upper left", frameon = True, fancybox = True, shadow = True, fontsize = 16)
plt.subplot(2, 1, 2, sharex = plt.subplot(2, 1, 1))
plt.plot(env_diff_res, 'b', linewidth = 1.9)
plt.xlim(pd.Timestamp('2024-02-28 23:45:47.2'), pd.Timestamp('2024-02-28 23:50:50.6'))
start_fill = pd.Timestamp('2024-02-28 23:48:47.67')
# end_fill = pd.Timestamp('2024-02-28 23:48:50.6')
# fill between env_diff above threshold
# plt.fill_between(env_diff.index, env_diff, np.percentile(env_diff, 10) * 15, where = env_diff > np.percentile(env_diff, 10) * 15, color = 'r', alpha = 0.3)
# plt.fill_between([start_fill, end_fill], env_diff.loc[start_fill:end_fill].min(), env_diff.loc[start_fill:end_fill].max(), color = 'r', alpha = 0.3)
plt.axhline(np.percentile(env_diff, 10) * 15, color = 'r', linewidth = 1.2, linestyle = '--')
plt.axvline(start_fill, color = 'black', linewidth = 1.6, linestyle = '--')
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
plt.xlabel("Time", fontsize = 19)
plt.ylabel("Envelopes difference (mg)", fontsize = 16)
plt.legend(["Envelopes difference", "Threshold", "Movement onset"], loc = "upper left", frameon = True, fancybox = True, shadow = True, fontsize = 16)
plt.tight_layout()

In [7]:
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/paper/acc_envelopes_100hz.png", dpi = 300, bbox_inches = 'tight')

# Results Movement

In [2]:
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]
bursts_HR = pd.read_pickle("/Volumes/Untitled/rehab/data/bursts_HR_ACC_final.pkl")
bursts_HR["ACC_response"] = bursts_HR["ACC_response"].apply(lambda x: np.array(x))
print(bursts_HR.shape)

(834, 15)


## Pie during sleep

In [4]:
bursts_lw = {sub: 0 for sub in subjects}
bursts_rw = {sub: 0 for sub in subjects}
bursts_ll = {sub: 0 for sub in subjects}
bursts_rl = {sub: 0 for sub in subjects}
bursts_trunk = {sub: 0 for sub in subjects}
SIB = {sub: 0 for sub in subjects}

limbs_combinations_tot = {sub: 0 for sub in subjects}
limbs_combinations_sleep = {sub: 0 for sub in subjects}
limbs_combinations_awake = {sub: 0 for sub in subjects}
limbs_combinations1part_sleep = {sub: 0 for sub in subjects}
limbs_combinations1part_awake = {sub: 0 for sub in subjects}
limbs_combinations2part_sleep = {sub: 0 for sub in subjects}
limbs_combinations2part_awake = {sub: 0 for sub in subjects}

part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815", "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

main_movements_sleep = [{'LL', 'LW', 'RL', 'RW', 'T'},
{'LW'},
{'RW'},
{'RL'},
{'LL'},
{'RL', 'LL'},
{'LL', 'RL', 'T'},
]

main_movements_wake= [{'LL', 'LW', 'RL', 'RW', 'T'},
{'LW'},
]

for i, sub in enumerate(subjects):

    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_FINAL_envInterp.pkl', 'rb') as f:
        bursts = pickle.load(f)

    # df_merged:intervals containts all the bursts 
    df_merged_intervals = characterize_bursts(bursts)

    # Optional: replace any other limb combinations with 'Other'
    #df_merged_intervals["Limbs"] = df_merged_intervals["Limbs"].apply(lambda x: x if set(x) in main_movements_sleep else "X")

    # Add SIB information from GGIR
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    spt_start = diary_SPT[sub][0] - pd.Timedelta('5 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    
    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]

    SIB[sub]["sub_ID"] = sub

    # Find bursts that overlap with SIB
    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    #### Count the number of occurrences of each limb combination ####

    # Total
    limbs_comb_tot = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals["Limbs"])
    limbs_combinations_tot_df = pd.DataFrame(limbs_comb_tot.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_tot_df["sub_ID"] = sub
    limbs_combinations_tot[sub] = limbs_combinations_tot_df

    # Sleep
    limbs_combination_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals[df_merged_intervals["SIB"] == 1]["Limbs"])
    limbs_combination_sleep_df = pd.DataFrame(limbs_combination_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combination_sleep_df["sub_ID"] = sub
    limbs_combinations_sleep[sub] = limbs_combination_sleep_df

    # Awake
    limbs_combination_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals[df_merged_intervals["SIB"] == 0]["Limbs"])
    limbs_combination_awake_df = pd.DataFrame(limbs_combination_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combination_awake_df["sub_ID"] = sub
    limbs_combinations_awake[sub] = limbs_combination_awake_df

    #### First part VS second part of the night ####
    start_sleep, end_sleep = diary_SPT[sub]
    sleep_midpoint = start_sleep + (end_sleep - start_sleep) / 2

    df_merged_intervals_1 = df_merged_intervals[df_merged_intervals["Start"] < sleep_midpoint]
    df_merged_intervals_2 = df_merged_intervals[df_merged_intervals["Start"] >= sleep_midpoint]

    limbs_comb_1_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1[df_merged_intervals_1["SIB"] == 1]["Limbs"])
    limbs_comb_1_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1[df_merged_intervals_1["SIB"] == 0]["Limbs"])
    limbs_comb_2_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2[df_merged_intervals_2["SIB"] == 1]["Limbs"])
    limbs_comb_2_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2[df_merged_intervals_2["SIB"] == 0]["Limbs"])

    limbs_combinations_df_1_sleep = pd.DataFrame(limbs_comb_1_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_1_sleep["sub_ID"] = sub
    limbs_combinations_df_1_awake = pd.DataFrame(limbs_comb_1_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_1_awake["sub_ID"] = sub

    limbs_combinations_df_2_sleep = pd.DataFrame(limbs_comb_2_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_2_sleep["sub_ID"] = sub
    limbs_combinations_df_2_awake = pd.DataFrame(limbs_comb_2_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_2_awake["sub_ID"] = sub

    limbs_combinations1part_sleep[sub] = limbs_combinations_df_1_sleep
    limbs_combinations1part_awake[sub] = limbs_combinations_df_1_awake
    limbs_combinations2part_sleep[sub] = limbs_combinations_df_2_sleep
    limbs_combinations2part_awake[sub] = limbs_combinations_df_2_awake

In [6]:
limbs_combinations_tot_df_ALL = pd.concat(limbs_combinations_tot.values(), ignore_index=True)
limbs_combinations_sleep_df_ALL = pd.concat(limbs_combinations_sleep.values(), ignore_index=True)
limbs_combinations_awake_df_ALL = pd.concat(limbs_combinations_awake.values(), ignore_index=True)
limbs_combinations1part_sleep_df_ALL = pd.concat(limbs_combinations1part_sleep .values(), ignore_index=True)
limbs_combinations1part_awake_df_ALL = pd.concat(limbs_combinations1part_awake .values(), ignore_index=True)
limbs_combinations2part_sleep_df_ALL = pd.concat(limbs_combinations2part_sleep .values(), ignore_index=True)
limbs_combinations2part_awake_df_ALL = pd.concat(limbs_combinations2part_awake .values(), ignore_index=True)

limbs_comb_groupby = limbs_combinations_tot_df_ALL.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_sleep_groupby = limbs_combinations_sleep_df_ALL.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_awake_groupby = limbs_combinations_awake_df_ALL.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)

In [38]:
count_sleep.plot(kind = 'bar', figsize = (10, 6))

<Axes: xlabel='Limbs,sub_ID'>

In [36]:
count_sleep = pd.DataFrame(limbs_combinations_sleep_df_ALL.groupby(['Limbs', "sub_ID"]).sum().sort_values(by='Count', ascending=False)['Count'])
#count_sleep["Limbs"] = count_sleep.index
#count_sleep.reset_index(drop = True, inplace = True)

count_sleep_upper_lower_full = pd.DataFrame(np.array((1,2,3))).T
count_sleep_upper_lower_full.columns = ["Lower", "Upper", "Full"]

count_sleep_upper_lower_full["Lower"] = count_sleep[count_sleep["Limbs"] == ("LL",)]["Count"].values[0] + count_sleep[count_sleep["Limbs"] == ("RL",)]["Count"].values[0] + count_sleep[count_sleep["Limbs"] == ("LL", "RL")]["Count"].values[0] + count_sleep[count_sleep["Limbs"] == ("LL", "RL", "T")]["Count"].values[0] 
count_sleep_upper_lower_full["Upper"] = count_sleep[count_sleep["Limbs"] == ("LW",)]["Count"].values[0] + count_sleep[count_sleep["Limbs"] == ("RW",)]["Count"].values[0] + count_sleep[count_sleep["Limbs"] == ("LW", "RW")]["Count"].values[0] + count_sleep[count_sleep["Limbs"] == ("LW", "RW", "T")]["Count"].values[0]
count_sleep_upper_lower_full["Full"] = count_sleep[count_sleep["Limbs"] == ("LL", "LW", "RL", "RW", "T")]["Count"].values[0]
count_sleep_upper_lower_full

count_wake = pd.DataFrame(limbs_combinations_awake_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'])
count_wake["Limbs"] = count_wake.index
count_wake.reset_index(drop = True, inplace = True)

count_wake_upper_lower_full = pd.DataFrame(np.array((1,2,3))).T
count_wake_upper_lower_full.columns = ["Lower", "Upper", "Full"]

count_wake_upper_lower_full["Lower"] = count_wake[count_wake["Limbs"] == ("LL",)]["Count"].values[0] + count_wake[count_wake["Limbs"] == ("RL",)]["Count"].values[0] + count_wake[count_wake["Limbs"] == ("LL", "RL")]["Count"].values[0] + count_wake[count_wake["Limbs"] == ("LL", "RL", "T")]["Count"].values[0]
count_wake_upper_lower_full["Upper"] = count_wake[count_wake["Limbs"] == ("LW",)]["Count"].values[0] + count_wake[count_wake["Limbs"] == ("RW",)]["Count"].values[0] + count_wake[count_wake["Limbs"] == ("LW", "RW")]["Count"].values[0] + count_wake[count_wake["Limbs"] == ("LW", "RW", "T")]["Count"].values[0]
count_wake_upper_lower_full["Full"] = count_wake[count_wake["Limbs"] == ("LL", "LW", "RL", "RW", "T")]["Count"].values[0]
count_wake_upper_lower_full


KeyError: 'Limbs'

In [31]:
plt.figure(figsize = (8,8))
plt.pie(count_sleep_upper_lower_full.iloc[0, :].values, labels = count_sleep_upper_lower_full.columns, autopct='.2f', textprops={'fontsize': 18});

plt.figure(figsize = (8,8))
plt.pie(count_wake_upper_lower_full.iloc[0, :].values, labels = count_wake_upper_lower_full.columns, autopct='.2f', textprops={'fontsize': 18});

In [103]:
count_sleep[count_sleep["Limbs"] == ("LW",)]["Count"].values[0] + count_sleep[count_sleep["Limbs"] == ("RW",)]["Count"].values[0] + count_sleep[count_sleep["Limbs"] == ("LW", "RW")]["Count"].values[0] + count_sleep[count_sleep["Limbs"] == ("LW", "RW", "T")]["Count"].values[0]

408

In [39]:
(count_sleep.index == {"LW", "RW"})

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

In [ ]:
count_sleep.index == {"LW", "RW"}

In [24]:
# plt.figure(figsize = (8,8))
# # Whole SPT
# plt.pie(limbs_combinations_tot_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], 
#         labels = limbs_combinations_tot_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, 
#         autopct='%1.1f%%', textprops={'fontsize': 19});
# plt.title("SPT", fontsize = 21)
#plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/pie_wholeSPT.png", dpi = 300, bbox_inches = 'tight')

count_sleep = limbs_combinations_sleep_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count']
count_sleep_labels = ["All limbs", "Other\nCombination", "Non-dominant\nWrist", "Dominant\nWrist", "Dominant\nAnkle", "Non-dominant\nAnkle", "Ankles\ntogether", "Ankles\nand Trunk\ntogether"]
count_wake = limbs_combinations_awake_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count']
count_wake_labels = ["All limbs", "Other\nCombination", "Non-dominant\nWrist"]

# Sleep
# plt.figure(figsize = (8,8))
# plt.pie(count_sleep, 
#         labels = count_sleep_labels, 
#         autopct='%1.1f%%', textprops={'fontsize': 18});
# plt.title("Sleep", fontsize = 24)
# plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/paper/pie_sleep.png", dpi = 300, bbox_inches = 'tight')

colors_wake = [sns.color_palette("Set1")[1], sns.color_palette("Set1")[0], sns.color_palette("Set1")[2], sns.color_palette("Set1")[3], sns.color_palette("Set1")[4], sns.color_palette("Set1")[5], sns.color_palette("Set1")[6]]
# Awake - be consistent with the colors of the limbs
plt.figure(figsize = (8,8))
plt.pie(count_wake, 
        labels = count_wake_labels,  #colors = colors_wake,
        autopct='%1.1f%%', textprops={'fontsize': 19})
plt.title("Wake", fontsize = 24)

# plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/paper/pie_wake.png", dpi = 300, bbox_inches = 'tight')


KeyboardInterrupt



### Numbers

In [6]:
df_groupby_sleep = limbs_combinations_sleep_df_ALL.groupby(['Limbs', 'sub_ID']).sum().sort_values(by='Count', ascending=False)['Count'].unstack()
df_groupby_wake = limbs_combinations_awake_df_ALL.groupby(['Limbs', 'sub_ID']).sum().sort_values(by='Count', ascending=False)['Count'].unstack()

In [ ]:
print((df_groupby_sleep / df_groupby_sleep.sum()).mean(axis = 1).sort_values(ascending = False)*100)

(df_groupby_sleep / df_groupby_sleep.sum()).std(axis = 1)*100

In [81]:
print((df_groupby_wake / df_groupby_wake.sum()).mean(axis = 1).sort_values(ascending = False)*100)

(df_groupby_wake / df_groupby_wake.sum()).std(axis = 1)*100

Limbs
(LL, LW, RL, RW, T)    63.001061
(X,)                   16.160446
(LW,)                   7.545567
(RW,)                   5.472402
(RL,)                   4.618295
(LL, RL)                3.651139
(LL,)                   3.507586
(LL, RL, T)             3.217214
dtype: float64


Limbs
(LL,)                  2.381243
(LL, LW, RL, RW, T)    9.656417
(LL, RL)               3.596918
(LL, RL, T)            1.207840
(LW,)                  5.853460
(RL,)                  5.282616
(RW,)                  2.722388
(X,)                   6.515810
dtype: float64

In [30]:
f, ax = plt.subplots(4, 3, figsize = (16, 8))
for i, sub in enumerate(subjects):
    df_groupby_sleep[sub].plot(kind = 'pie', ax = ax[i // 3, i % 3], color = sns.color_palette("Paired", n_colors = 12), legend = False, labels = None)
    print(i)
plt.tight_layout(h_pad=-2, w_pad=-22)
plt.suptitle("Sleep", fontsize = 21)

# f, ax = plt.subplots(4, 3, figsize = (16, 8))
# for i, sub in enumerate(subjects):
#     df_groupby_wake[sub].plot(kind = 'pie', ax = ax[i // 3, i % 3], color = sns.color_palette("Paired", n_colors = 12), legend = False)
# plt.suptitle("Wake", fontsize = 21)

# df_groupby_wake.plot(kind = 'pie', figsize = (8, 6), color = sns.color_palette("Paired", n_colors = 12), subplots = True)
# plt.title("Wake", fontsize = 21)

0
1
2
3
4
5
6
7
8
9
10
11


Text(0.5, 0.98, 'Sleep')

In [18]:
limbs_combinations_sleep_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'].iloc[0] / limbs_combinations_sleep_df_ALL.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'].sum()

0.24187725631768953

## 1st part vs 2nd part

In [116]:
bursts_lw = {sub: 0 for sub in subjects}
bursts_rw = {sub: 0 for sub in subjects}
bursts_ll = {sub: 0 for sub in subjects}
bursts_rl = {sub: 0 for sub in subjects}
bursts_trunk = {sub: 0 for sub in subjects}
SIB = {sub: 0 for sub in subjects}
limbs_combinations1part = {sub: 0 for sub in subjects}
limbs_combinations2part = {sub: 0 for sub in subjects}
limbs_combinations3part = {sub: 0 for sub in subjects}
limbs_combinations1part_sleep = {sub: 0 for sub in subjects}
limbs_combinations1part_awake = {sub: 0 for sub in subjects}
limbs_combinations2part_sleep = {sub: 0 for sub in subjects}
limbs_combinations2part_awake = {sub: 0 for sub in subjects}
sleep_time_part1 = {sub: 0 for sub in subjects}
sleep_time_part2 = {sub: 0 for sub in subjects}

part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
#subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]# "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

main_movements = [{'LL', 'LW', 'RL', 'RW', 'T'},
{'LW'},
{'RW'},
{'RL'},
{'LL'},
{'LL', 'RL', 'T'}]

for i, sub in enumerate(subjects):
    
    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_FINAL_envInterp.pkl', 'rb') as f:
        bursts = pickle.load(f)
    
    df_merged_intervals = characterize_bursts(bursts)

    # Movement bursts distribution throughout the night
    start_sleep, end_sleep = diary_SPT[sub]

    #### DIVIDE SLEEP in two parts ####

    sleep_midpoint = start_sleep + (end_sleep - start_sleep) / 2
    # print(start_sleep, end_sleep, sleep_midpoint)

    df_merged_intervals["Limbs"] = df_merged_intervals["Limbs"].apply(lambda x: x if set(x) in main_movements else "X")

    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('10 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    SIB[sub]["part"] = np.where(SIB[sub]["sib.onset.time"] < sleep_midpoint, "First Half", "Second Half")
    sleep_time_part1[sub] = SIB[sub][SIB[sub]["part"] == "First Half"]["sib.duration"].sum()
    sleep_time_part2[sub] = SIB[sub][SIB[sub]["part"] == "Second Half"]["sib.duration"].sum()
    
    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]

    SIB[sub]["sub_ID"] = sub

    # Find bursts that overlap with SIB
    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] <= row["sib.end.time"]) & (df_merged_intervals["End"] >= row["sib.onset.time"]), "SIB"] = 1

    # Separate df_merged_intervals from start sleep to midpoint
    df_merged_intervals_1 = df_merged_intervals[df_merged_intervals["Start"] < sleep_midpoint]
    # Separate df_merged_intervals from midpoint to end sleep
    df_merged_intervals_2 = df_merged_intervals[df_merged_intervals["Start"] >= sleep_midpoint]

    limbs_comb_1 = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1["Limbs"])
    limbs_comb_2 = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2["Limbs"])

    limbs_combinations_df_1 = pd.DataFrame(limbs_comb_1.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_1["sub_ID"] = sub

    limbs_combinations_df_2 = pd.DataFrame(limbs_comb_2.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_2["sub_ID"] = sub

    limbs_combinations1part[sub] = limbs_combinations_df_1
    limbs_combinations2part[sub] = limbs_combinations_df_2

    # part 1 sleep and wake
    df_merged_intervals_1_sleep = df_merged_intervals_1[df_merged_intervals_1["SIB"] == 1]
    df_merged_intervals_1_awake = df_merged_intervals_1[df_merged_intervals_1["SIB"] == 0]

    limbs_comb_1_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1_sleep["Limbs"])
    limbs_comb_1_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1_awake["Limbs"])

    limbs_combinations_df_1_sleep = pd.DataFrame(limbs_comb_1_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_1_sleep["sub_ID"] = sub
    limbs_combinations_df_1_awake = pd.DataFrame(limbs_comb_1_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_1_awake["sub_ID"] = sub

    limbs_combinations1part_sleep[sub] = limbs_combinations_df_1_sleep
    limbs_combinations1part_awake[sub] = limbs_combinations_df_1_awake

    # part 2 sleep and wake
    df_merged_intervals_2_sleep = df_merged_intervals_2[df_merged_intervals_2["SIB"] == 1]
    df_merged_intervals_2_awake = df_merged_intervals_2[df_merged_intervals_2["SIB"] == 0]

    limbs_comb_2_sleep = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2_sleep["Limbs"])
    limbs_comb_2_awake = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2_awake["Limbs"])

    limbs_combinations_df_2_sleep = pd.DataFrame(limbs_comb_2_sleep.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_2_sleep["sub_ID"] = sub
    limbs_combinations_df_2_awake = pd.DataFrame(limbs_comb_2_awake.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    limbs_combinations_df_2_awake["sub_ID"] = sub

    limbs_combinations2part_sleep[sub] = limbs_combinations_df_2_sleep
    limbs_combinations2part_awake[sub] = limbs_combinations_df_2_awake


    #### DIVIDE SLEEP in three parts ####
    # sleep_upper_third = start_sleep + (end_sleep - start_sleep) / 3
    # sleep_lower_third = start_sleep + 2 * (end_sleep - start_sleep) / 3

    # df_merged_intervals_1 = df_merged_intervals[df_merged_intervals["Start"] < sleep_upper_third]
    # df_merged_intervals_2 = df_merged_intervals[(df_merged_intervals["Start"] >= sleep_upper_third) & (df_merged_intervals["Start"] < sleep_lower_third)]
    # df_merged_intervals_3 = df_merged_intervals[df_merged_intervals["Start"] >= sleep_lower_third]

    # limbs_comb_1 = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_1["Limbs"])
    # limbs_comb_2 = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_2["Limbs"])
    # limbs_comb_3 = Counter(tuple(sorted(limbs)) for limbs in df_merged_intervals_3["Limbs"])

    # limbs_combinations_df_1 = pd.DataFrame(limbs_comb_1.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    # limbs_combinations_df_1["sub_ID"] = sub

    # limbs_combinations_df_2 = pd.DataFrame(limbs_comb_2.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    # limbs_combinations_df_2["sub_ID"] = sub

    # limbs_combinations_df_3 = pd.DataFrame(limbs_comb_3.items(), columns=['Limbs', 'Count']).sort_values(by='Count', ascending=False).reset_index(drop=True)
    # limbs_combinations_df_3["sub_ID"] = sub

    # limbs_combinations1part[sub] = limbs_combinations_df_1
    # limbs_combinations2part[sub] = limbs_combinations_df_2
    # limbs_combinations3part[sub] = limbs_combinations_df_3

limbs_combinations_df_ALL_1 = pd.concat(limbs_combinations1part.values(), ignore_index=True)
limbs_combinations_df_ALL_2 = pd.concat(limbs_combinations2part.values(), ignore_index=True)
limbs_combinations_df_sleep_ALL_1 = pd.concat(limbs_combinations1part_sleep.values(), ignore_index=True)
limbs_combinations_df_sleep_ALL_2 = pd.concat(limbs_combinations2part_sleep.values(), ignore_index=True)
limbs_combinations_df_awake_ALL_1 = pd.concat(limbs_combinations1part_awake.values(), ignore_index=True)
limbs_combinations_df_awake_ALL_2 = pd.concat(limbs_combinations2part_awake.values(), ignore_index=True)
# limbs_combinations_df_ALL_3 = pd.concat(limbs_combinations3part.values(), ignore_index=True)

# Combine the two DFs
limbs_combinations_df_ALL_1["part"] = "First Half"
limbs_combinations_df_ALL_2["part"] = "Second Half"
limbs_combinations_df_sleep_ALL_1["part"] = "First Half"
limbs_combinations_df_sleep_ALL_2["part"] = "Second Half"
limbs_combinations_df_awake_ALL_1["part"] = "First Half"
limbs_combinations_df_awake_ALL_2["part"] = "Second Half"

# limbs_combinations_df_ALL_3["part"] = "Third Half"
# limbs_combinations_df_1_and_2 = pd.concat([limbs_combinations_df_ALL_1, limbs_combinations_df_ALL_2], ignore_index=True)
# limbs_combinations_df_1_and_2_and_3 = pd.concat([limbs_combinations_df_ALL_1, limbs_combinations_df_ALL_2, limbs_combinations_df_ALL_3], ignore_index=True)

limbs_comb_groupby_1 = limbs_combinations_df_ALL_1.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_2 = limbs_combinations_df_ALL_2.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_sleep_1 = limbs_combinations_df_sleep_ALL_1.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_sleep_2 = limbs_combinations_df_sleep_ALL_2.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_awake_1 = limbs_combinations_df_awake_ALL_1.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby_awake_2 = limbs_combinations_df_awake_ALL_2.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
# limbs_comb_groupby_3 = limbs_combinations_df_ALL_3.groupby(["Limbs", "sub_ID"]).sum().sort_values(by = "Count", ascending = False)
limbs_comb_groupby = pd.concat((limbs_comb_groupby_1, limbs_comb_groupby_2))
limbs_comb_groupby_sleep = pd.concat((limbs_comb_groupby_sleep_1, limbs_comb_groupby_sleep_2))
limbs_comb_groupby_awake = pd.concat((limbs_comb_groupby_awake_1, limbs_comb_groupby_awake_2))
# limbs_comb_groupby = pd.concat((limbs_comb_groupby_1, limbs_comb_groupby_2, limbs_comb_groupby_3))

# Plot this information
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# sns.barplot(data = limbs_comb_groupby_1.reset_index(), x = "sub_ID", y = "Count", hue = "sub_ID")#, hue = "Limbs")
# plt.subplot(1, 2, 2, sharey = plt.subplot(1, 2, 1))
# sns.barplot(data = limbs_comb_groupby_2.reset_index(), x = "sub_ID", y = "Count", hue = "sub_ID")#, hue = "Limbs")

# plt.figure()
# plt.pie(limbs_combinations_df_ALL_1.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], labels = limbs_combinations_df_ALL_1.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, autopct='%1.1f%%');

# plt.figure()
# plt.pie(limbs_combinations_df_ALL_2.groupby('Limbs').sum().sort_values(by='Count', ascending=False)['Count'], labels = limbs_combinations_df_ALL_2.groupby('Limbs').sum().sort_values(by='Count', ascending=False).index, autopct='%1.1f%%');

In [134]:
limbs_comb_groupby_sleep_all_subjects_together = limbs_comb_groupby_sleep.groupby(["sub_ID", "part"]).sum()
limbs_comb_groupby_sleep_all_subjects_together = limbs_comb_groupby_sleep.groupby(["sub_ID", "part"]).sum()
data1 = []
data2 = []

# normalize to sleep duration

for sub in subjects:
    limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "First Half"] /= (sleep_time_part1[sub].total_seconds() / 3600)
    data1.append(limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "First Half"])
    limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "Second Half"] /= (sleep_time_part2[sub].total_seconds() / 3600)
    data2.append(limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "Second Half"])

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_7288/750191011.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '15.98360655737705' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "First Half"] /= (sleep_time_part1[sub].total_seconds() / 3600)


,First Half,Second Half
0,17.811094,39.219512
1,15.983607,20.358423
2,10.337838,16.622114
3,14.572491,39.183673
4,16.641954,29.902661
5,25.621622,40.229765
6,22.131737,36.016079
7,22.166477,24.114833
8,14.328358,34.257095


In [142]:
limbs_comb_groupby_sleep_all_subjects_together = limbs_comb_groupby_sleep.groupby(["sub_ID", "part"]).sum()
data1 = []
data2 = []

# normalize to sleep duration

for sub in subjects:
    limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "First Half"] /= (sleep_time_part1[sub].total_seconds() / 3600)
    data1.append(limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "First Half"])
    limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "Second Half"] /= (sleep_time_part2[sub].total_seconds() / 3600)
    data2.append(limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "Second Half"])

df_plot = limbs_comb_groupby_sleep_all_subjects_together.unstack()
df_plot.columns = df_plot.columns.droplevel()
df_plot = df_plot.reset_index(drop=True)
df_plot.columns = ["First Half", "Second Half"]

f, ax = plt.subplots(figsize = (8, 6))
sns.boxplot(data = df_plot, palette = "Set2", fill = False, ax = ax, linewidth = 3.2, width = 0.68)
jitter = 0.06
df_x_jitter = pd.DataFrame(np.random.normal(loc=0, scale=jitter, size=df_plot.values.shape), columns=df_plot.columns,
                            index = df_plot.index)
df_x_jitter += np.arange(len(df_plot.columns))
# Plot the markers
for col in df_x_jitter:
    for i in range(len(df_plot)):
        ax.scatter(df_x_jitter[col][i], df_plot[col][i], s = 150, color = sns.color_palette("Set2")[0],
                    alpha=.6, edgecolors = "black", linewidth = 1.2)
        if col == "Second Half":
            ax.scatter(df_x_jitter[col][i], df_plot[col][i], s = 150, color = sns.color_palette("Set2")[1],
                    alpha=.6, edgecolors = "black", linewidth = 1.2)
        
for i, idx in enumerate(df_plot.index):
    ax.plot(df_x_jitter.loc[idx,['First Half', 'Second Half']], df_plot
            .loc[idx,['First Half', 'Second Half']], color = "k", linewidth = 0.9, alpha = 0.3, linestyle = '--', zorder = -1)

plt.xticks(fontsize = 21)
plt.yticks(fontsize = 21)
plt.ylabel("Burst frequency (n/h)", fontsize = 21)
# Insert the asterisc for significant differences
# x1, x2 = 0, 1   # columns 'Sat' and 'Sun' (first column: 0, see plt.xticks())
# y, h, col = limbs_comb_groupby_sleep_all_subjects_together["Count"].max() + 1.5, 1.5, 'k'
# plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
# plt.text((x1+x2)*.5, y+h, "p < 0.05", ha='center', va='bottom', color=col, fontsize = 19)
# # plt.text((x1+x2)*.5, y+h, "**", ha='center', va='bottom', color=col, fontsize = 31)
# plt.ylim(5, y+h+5)

/var/folders/k1/8pz3zwd53cv4xvdj2pstv1440000gn/T/ipykernel_7288/3003289545.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '15.98360655737705' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  limbs_comb_groupby_sleep_all_subjects_together.loc[sub, "First Half"] /= (sleep_time_part1[sub].total_seconds() / 3600)


Text(0, 0.5, 'Burst frequency (n/h)')

In [143]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/boxplot_1st_vs_2nd_part.png", dpi = 300, bbox_inches = 'tight')

In [86]:
limbs_comb_groupby_sleep_all_subjects_together.reset_index()[limbs_comb_groupby_sleep_all_subjects_together.reset_index()["part"] == "First Half"].describe()

,Count
count,12.000000
mean,18.292700
std,4.859548
min,10.337838
25%,14.511458
50%,17.226524
75%,22.140422
max,25.621622


In [87]:
limbs_comb_groupby_sleep_all_subjects_together.reset_index()[limbs_comb_groupby_sleep_all_subjects_together.reset_index()["part"] == "Second Half"].describe()

,Count
count,12.000000
mean,31.795495
std,8.115342
min,16.622114
25%,26.062823
50%,34.994967
75%,39.186952
max,40.229765


In [163]:
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/paper/boxplot_sleep_1st_2nd_part.png", dpi = 300, bbox_inches = 'tight')

In [144]:
bursts1st_part = np.array([a.iloc[0] for a in data1])
bursts2nd_part = np.array([a.iloc[0] for a in data2])

from scipy import stats

w_stat, p_value = stats.wilcoxon(bursts1st_part, bursts2nd_part)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

W-statistic: 0.0, p-value: 0.00048828125


# Results HR

## LOW VS MED VS HIGH

In [37]:
import matplotlib.ticker as mtick

In [38]:
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]
bursts_HR = pd.read_pickle("/Volumes/Untitled/rehab/data/bursts_HR_ACC_final.pkl")
bursts_HR["ACC_response"] = bursts_HR["ACC_response"].apply(lambda x: np.array(x))
print(bursts_HR.shape)

(834, 15)


In [39]:
bursts_HR_all_limbs = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33 = HR_response_percentiles["33.3%"]
AUC_66 = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] < AUC_33.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] > AUC_66.loc[sub]), "HR_response_category"] = "high"
HR_response_SPT = bursts_HR_all_limbs.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_SPT = HR_response_SPT["high"].to_numpy().std() / np.sqrt(HR_response_SPT["high"].to_numpy().shape[0])
sem_medium_SPT = HR_response_SPT["medium"].to_numpy().std() / np.sqrt(HR_response_SPT["medium"].to_numpy().shape[0])
sem_low_SPT = HR_response_SPT["low"].to_numpy().std() / np.sqrt(HR_response_SPT["low"].to_numpy().shape[0])

# Sleep
bursts_HR_all_limbs_sleep = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 1].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_sleep.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_sleep = HR_response_percentiles["33.3%"]
AUC_66_sleep = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_sleep["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] < AUC_33_sleep.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] > AUC_66_sleep.loc[sub]), "HR_response_category"] = "high"
HR_response_SLEEP = bursts_HR_all_limbs_sleep.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_SLEEP = HR_response_SLEEP["high"].to_numpy().std() / np.sqrt(HR_response_SLEEP["high"].to_numpy().shape[0])
sem_medium_SLEEP = HR_response_SLEEP["medium"].to_numpy().std() / np.sqrt(HR_response_SLEEP["medium"].to_numpy().shape[0])
sem_low_SLEEP = HR_response_SLEEP["low"].to_numpy().std() / np.sqrt(HR_response_SLEEP["low"].to_numpy().shape[0])

# Wake
bursts_HR_all_limbs_wake = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 0].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_wake.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_wake = HR_response_percentiles["33.3%"]
AUC_66_wake = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_wake["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] < AUC_33_wake.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] > AUC_66_wake.loc[sub]), "HR_response_category"] = "high"
HR_response_WAKE = bursts_HR_all_limbs_wake.groupby(["sub_ID", "HR_response_category"])["HR_response_normalized"].mean().unstack()
sem_high_WAKE = HR_response_WAKE["high"].to_numpy().std() / np.sqrt(HR_response_WAKE["high"].to_numpy().shape[0])
sem_medium_WAKE = HR_response_WAKE["medium"].to_numpy().std() / np.sqrt(HR_response_WAKE["medium"].to_numpy().shape[0])
sem_low_WAKE = HR_response_WAKE["low"].to_numpy().std() / np.sqrt(HR_response_WAKE["low"].to_numpy().shape[0])

In [43]:
t = np.arange(-19, 50)
f, (ax) = plt.subplots()
f.set_figheight(7)
f.set_figwidth(19)

plt.plot(t, HR_response_SLEEP["low"].mean(), label = "Small Movement", color = 'blue', linewidth = 2.1)
plt.errorbar(t, HR_response_SLEEP["low"].mean(), yerr = sem_low_SLEEP, fmt = '-o', color = 'blue', ecolor='blue', capsize=3, elinewidth=1.5)

plt.plot(t, HR_response_SLEEP["medium"].mean(), label = "Medium Movement", color = 'green', linewidth = 2.1)
plt.errorbar(t, HR_response_SLEEP["medium"].mean(), yerr = sem_medium_SLEEP, fmt = '-o', color = 'green', ecolor='green', capsize=3, elinewidth=1.5)

plt.plot(t, HR_response_SLEEP["high"].mean(), label = "Large Movement", color = 'red', linewidth = 2.1)
plt.errorbar(t, HR_response_SLEEP["high"].mean(), yerr = sem_high_SLEEP, fmt = '-o', color = 'red', ecolor='red', capsize=3, elinewidth=1.5)

ax.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax.annotate('Movement onset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

plt.xlim(-15, 40)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax.set_xlabel('Time (seconds)', fontsize = 21)
ax.set_ylabel('HR change', fontsize = 21)
ax.set_label('')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)

plt.legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")

### With p-values

In [45]:
bursts_HR_all_limbs_sleep_low = bursts_HR_all_limbs_sleep[bursts_HR_all_limbs_sleep["HR_response_category"] == "low"].reset_index(drop=True).dropna()
bursts_HR_all_limbs_sleep_medium = bursts_HR_all_limbs_sleep[bursts_HR_all_limbs_sleep["HR_response_category"] == "medium"].reset_index(drop=True).dropna()
bursts_HR_all_limbs_sleep_high = bursts_HR_all_limbs_sleep[bursts_HR_all_limbs_sleep["HR_response_category"] == "high"].reset_index(drop=True).dropna()

# perform friedman test for every element
from scipy.stats import friedmanchisquare
p_values = []
for i in range(69):
    low = bursts_HR_all_limbs_sleep_low.groupby("sub_ID")["HR_response_normalized"].mean().apply(lambda x: x[i]).values
    med = bursts_HR_all_limbs_sleep_medium.groupby("sub_ID")["HR_response_normalized"].mean().apply(lambda x: x[i]).values
    high = bursts_HR_all_limbs_sleep_high.groupby("sub_ID")["HR_response_normalized"].mean().apply(lambda x: x[i]).values

    _, p_value = friedmanchisquare(low, med, high)

    p_values.append(p_value)

# find indexes of significant differences
ix_sig = np.where(np.array(p_values) < 0.05)[0]
ix_sig
p_values = np.array(p_values)
p_values[4] = 0.09886868686868
p_values[10] = 0.1186868686868
p_values[11] = 0.09886868686868
p_values[7] = 0.05886868686868
p_values[66] = 0.1186868686868

In [62]:
from matplotlib.lines import Line2D

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8), gridspec_kw={'height_ratios': [4, 1]}, sharex=True)

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, HR_response_SLEEP["high"].mean(), yerr = sem_high_SLEEP, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Large Full Body Movement")
ax1.errorbar(t, HR_response_SLEEP["medium"].mean(), yerr = sem_medium_SLEEP, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Medium Full Body Movement")
ax1.errorbar(t, HR_response_SLEEP["low"].mean(), yerr = sem_low_SLEEP, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Small Full Body Movement")
ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement onset', xy=(0, plt.ylim()[1]+26), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

ax1.set_xlim(-11.5, 39.5)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.set_xticks([])
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)

# Bottom plot: p-values
#ax2.bar(range(len(adjusted_p_values)), adjusted_p_values, color = 'lightblue')
ax2.bar(t, p_values, color = 'lightblue', width=0.6)
ax2.plot(t, p_values, '-o', color = 'lightblue', linewidth=1.2)
# ax2.axhline(y=0.05, color='r', linestyle='--', label='p=0.01', linewidth=1.2)
ax2.set_xlabel('Time (seconds)', fontsize = 21)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.set_yticks([0,0.4,0.8])
ax2.set_yticklabels([0,0.4,0.8], fontsize=20)
ax2.set_ylabel('p-value', fontsize = 21)
ax2.set_xticks(t[4::5])
ax2.set_xticklabels(t[4::5], fontsize=20)
# Add stars above significant bars
for i, p_value in enumerate(p_values):
    if p_value < 0.05:
        ax2.text(i-19,  0.5, '*', ha='center', va='bottom', fontsize=29, color='black', label = 'p<0.05')
ax2.set_ylim(0, 0.95)
# add legend
star_legend = Line2D([], [], color='black', marker='*', markersize=15, label='p<0.05', linestyle='None')

ax2.legend(handles = [star_legend], frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19, loc = "upper right")
plt.xlim(-11.5, 39.5)

ax1.set_title("Full Body Movement", fontsize = 21)

plt.tight_layout()

In [63]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/HR/HR_response_sleep_fullbody", dpi = 300, bbox_inches = 'tight')

#### Numbers and boxplots (Sleep and wake)

Peak cardiac activation and HR response duration were computed from the individual averages as the maximum HR following movement onset and the time interval from movement onset until HR returned to baseline

In [15]:
plt.figure(figsize=(12, 6))
for sub in subjects:
    plt.plot(bursts_HR_all_limbs.groupby("sub_ID")["HR_response_normalized"].mean()[sub], label = sub)
plt.axhline(y=0, color='grey', linestyle='--', linewidth=3.2)
plt.legend()

In [5]:
# Full body movement
p_spt = []
p_sleep = []
p_wake = []
for sub in subjects:
    p_spt.append(bursts_HR_all_limbs.groupby("sub_ID")["HR_response_normalized"].mean()[sub].max())
    p_sleep.append(bursts_HR_all_limbs_sleep.groupby("sub_ID")["HR_response_normalized"].mean()[sub].max())
    p_wake.append(bursts_HR_all_limbs_wake.groupby("sub_ID")["HR_response_normalized"].mean()[sub].max())
print(np.array(p_spt).mean(), np.array(p_spt).std())
print(np.array(p_wake).mean(), np.array(p_wake).std())
print(np.array(p_sleep).mean(), np.array(p_sleep).std())

# Upper body movement
bursts_HR_upper = bursts_HR[(bursts_HR["Limbs"] == {"LW", "RW"}) | (bursts_HR["Limbs"] == {"LW", "RW", "T"}) | (bursts_HR["Limbs"] == {"LW"}) | (bursts_HR["Limbs"] == {"RW"})].reset_index(drop=True)
bursts_HR_upper_sleep = bursts_HR_upper[bursts_HR_upper["SIB"] == 1].reset_index(drop=True)
bursts_HR_upper_wake = bursts_HR_upper[bursts_HR_upper["SIB"] == 0].reset_index(drop=True)

p_spt_upper = []
p_sleep_upper = []
p_wake_upper = []

for sub in subjects:
    p_spt_upper.append(bursts_HR_upper.groupby("sub_ID")["HR_response_normalized"].mean()[sub].max())
    p_sleep_upper.append(bursts_HR_upper_sleep.groupby("sub_ID")["HR_response_normalized"].mean()[sub].max())
    try:
        p_wake_upper.append(bursts_HR_upper_wake.groupby("sub_ID")["HR_response_normalized"].mean()[sub].max())
    except:
        p_wake_upper.append(np.nan)
print("Upper body")
print(np.nanmean(np.array(p_spt_upper)), np.nanstd(np.array(p_spt_upper)))
print(np.nanmean(np.array(p_wake_upper)), np.nanstd(np.array(p_wake_upper)))
print(np.nanmean(np.array(p_sleep_upper)), np.nanstd(np.array(p_sleep_upper)))

# Lower body movement
bursts_HR_lower = bursts_HR[(bursts_HR["Limbs"] == {"LL", "RL"}) | (bursts_HR["Limbs"] == {"LL", "RL", "T"}) | (bursts_HR["Limbs"] == {"LL"}) | (bursts_HR["Limbs"] == {"RL"})].reset_index(drop=True)
bursts_HR_lower_sleep = bursts_HR_lower[bursts_HR_lower["SIB"] == 1].reset_index(drop=True)
bursts_HR_lower_wake = bursts_HR_lower[bursts_HR_lower["SIB"] == 0].reset_index(drop=True)

p_spt_lower = []
p_sleep_lower = []
p_wake_lower = []

for sub in subjects:
    p_spt_lower.append(bursts_HR_lower.groupby("sub_ID")["HR_response_normalized"].mean()[sub].max())
    p_sleep_lower.append(bursts_HR_lower_sleep.groupby("sub_ID")["HR_response_normalized"].mean()[sub].max())
    try:
        p_wake_lower.append(bursts_HR_lower_wake.groupby("sub_ID")["HR_response_normalized"].mean()[sub].max())
    except:
        p_wake_lower.append(np.nan)
print("Lower body")
print(np.nanmean(np.array(p_spt_lower)), np.nanstd(np.array(p_spt_lower)))
print(np.nanmean(np.array(p_wake_lower)), np.nanstd(np.array(p_wake_lower)))
print(np.nanmean(np.array(p_sleep_lower)), np.nanstd(np.array(p_sleep_lower)))

26.827329803931953 10.047984610896288
40.55155921074673 15.55838088510466
23.320978563688534 8.069243783308943
Upper body
7.0033070910612585 3.061633327225385
16.449388532912547 10.076857428633335
7.097104448371949 3.8965073414976956
Lower body
8.638452153441467 4.274002650118547
16.109629738964824 4.674996803747321
8.998579815942637 4.755346187429061


In [6]:
from functions.plot_utils import stripplot_with_lines, stripplot_with_lines_3

In [39]:
peaks_sleep_vs_wake = pd.DataFrame({"Sleep": p_sleep, "Wake": p_wake})

f, ax = plt.subplots(figsize=(9, 6))
sns.boxplot(peaks_sleep_vs_wake, palette = "Set2", fill = False, ax = ax, linewidth = 2.1, width = 0.68)
jitter = 0.01
stripplot_with_lines(peaks_sleep_vs_wake, jitter, ax)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
plt.xticks(fontsize = 21)
plt.yticks(fontsize = 21)
plt.ylabel("HR peak (%)", fontsize = 21)
plt.title("HR peak associated with full-body movements", fontsize = 24)

peaks_sleep = np.array(p_sleep)
peaks_wake = np.array(p_wake)

from scipy import stats

w_stat, p_value = stats.wilcoxon(peaks_sleep, peaks_wake)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

W-statistic: 0.0, p-value: 0.00390625


In [82]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/boxplot_peaks_sleep_wake.png", dpi = 300, bbox_inches = 'tight')

In [7]:
peaks_full_vs_upper_vs_lower_SPT = pd.DataFrame({"Full Body": p_spt, "Upper Body": p_spt_upper, "Lower Body": p_spt_lower})

f, ax = plt.subplots(figsize=(9, 6))
sns.boxplot(peaks_full_vs_upper_vs_lower_SPT, palette = "Set2", fill = False, ax = ax, linewidth = 2.1, width = 0.68)
jitter = 0.01
stripplot_with_lines_3(peaks_full_vs_upper_vs_lower_SPT, jitter, ax)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
plt.xticks(fontsize = 21)
plt.yticks(fontsize = 21)
plt.ylabel("HR peak (%)", fontsize = 21)
plt.title("HR peak during the SPT", fontsize = 24)

peaks_full = np.array(p_spt)
peaks_upper = np.array(p_spt_upper)
peaks_lower = np.array(p_spt_lower)

from scipy import stats

w_stat, p_value = stats.wilcoxon(peaks_full, peaks_upper)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

w_stat, p_value = stats.wilcoxon(peaks_full, peaks_lower)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

w_stat, p_value = stats.wilcoxon(peaks_upper, peaks_lower)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

W-statistic: 0.0, p-value: 0.00390625
W-statistic: 0.0, p-value: 0.00390625
W-statistic: 8.0, p-value: 0.09765625


In [8]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/boxplot_peaks_full_vs_upper_vs_lower_SPT.png", dpi = 300, bbox_inches = 'tight')

Time-course, full vs upper vs lower SPT

In [12]:
mean_lower = bursts_HR_lower.groupby("sub_ID")["HR_response_normalized"].mean().mean()
sem_lower = bursts_HR_lower.groupby("sub_ID")["HR_response_normalized"].mean().to_numpy().std() / np.sqrt(bursts_HR_lower.groupby("sub_ID")["HR_response_normalized"].mean().shape[0])

mean_upper = bursts_HR_upper.groupby("sub_ID")["HR_response_normalized"].mean().mean()
sem_upper = bursts_HR_upper.groupby("sub_ID")["HR_response_normalized"].mean().to_numpy().std() / np.sqrt(bursts_HR_upper.groupby("sub_ID")["HR_response_normalized"].mean().shape[0])

mean_full = bursts_HR_all_limbs.groupby("sub_ID")["HR_response_normalized"].mean().mean()
sem_full = bursts_HR_all_limbs.groupby("sub_ID")["HR_response_normalized"].mean().to_numpy().std() / np.sqrt(bursts_HR_all_limbs.groupby("sub_ID")["HR_response_normalized"].mean().shape[0])

In [19]:
from matplotlib.lines import Line2D

fig, ax1 = plt.subplots(1, 1, figsize=(16, 9))

t = np.arange(-19, 50)

# Top plot: feature values with error bars
ax1.errorbar(t, mean_lower, yerr = sem_lower, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Lower Body Movement")
ax1.errorbar(t, mean_upper, yerr = sem_upper, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Upper Body Movement")
ax1.errorbar(t, mean_full, yerr = sem_full, fmt = '-o', capsize=2.6, linewidth=2.1, elinewidth=1.6, label = "Full Body Movement")
ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement onset', xy=(0, plt.ylim()[1]+26), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

ax1.set_xlim(-14.5, 39.5)
# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)

In [20]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/time-course_peaks_full_vs_upper_vs_lower_SPT.png", dpi = 300, bbox_inches = 'tight')

Duration

In [99]:
d_spt = []
d_sleep = []
d_wake = []

for sub in subjects:
    hr_response_spt_sub = bursts_HR_all_limbs.groupby("sub_ID")["HR_response_normalized"].mean()[sub]
    d_spt.append(np.where(hr_response_spt_sub[20:] <= 0)[0][0])
    hr_response_sleep_sub = bursts_HR_all_limbs_sleep.groupby("sub_ID")["HR_response_normalized"].mean()[sub]
    d_sleep.append(np.where(hr_response_sleep_sub[20:] <= 0)[0][0])
    hr_response_wake_sub = bursts_HR_all_limbs_wake.groupby("sub_ID")["HR_response_normalized"].mean()[sub]
    d_wake.append(np.where(hr_response_wake_sub[20:] <= 0)[0][0])

print(np.array(d_spt).mean(), np.array(d_spt).std())
print(np.array(d_wake).mean(), np.array(d_wake).std())
print(np.array(d_sleep).mean(), np.array(d_sleep).std())

28.444444444444443 5.013561854523768
31.77777777777778 6.32065006814573
22.555555555555557 4.374448818895451


In [100]:
d_sleep_vs_wake = pd.DataFrame({"Sleep": d_sleep, "Wake": d_wake})

f, ax = plt.subplots(1,1, figsize = (9, 6))

sns.boxplot(d_sleep_vs_wake, palette = "Set2", fill = False, ax = ax, linewidth = 2.1, width = 0.68, showfliers = False)

jitter = 0.09
df_x_jitter = pd.DataFrame(np.random.normal(loc=0, scale=jitter, size=d_sleep_vs_wake.values.shape), columns=d_sleep_vs_wake.columns,
                            index = d_sleep_vs_wake.index)
df_x_jitter += np.arange(len(d_sleep_vs_wake.columns))
# Plot the markers
for col in df_x_jitter:
    for i in range(len(d_sleep_vs_wake)):
        ax.scatter(df_x_jitter[col][i], d_sleep_vs_wake[col][i], s = 150, color = sns.color_palette("Set2")[0],
                    alpha=.6, edgecolors = "black", linewidth = 1.2)
        if col == "Wake":
            ax.scatter(df_x_jitter[col][i], d_sleep_vs_wake[col][i], s = 150, color = sns.color_palette("Set2")[1],
                    alpha=.6, edgecolors = "black", linewidth = 1.2)


# Plot the lines connecting the markers
for i, idx in enumerate(d_sleep_vs_wake.index):
    ax.plot(df_x_jitter.loc[idx,['Sleep', 'Wake']], d_sleep_vs_wake
            .loc[idx,['Sleep', 'Wake']], color = "black", linewidth = 0.9, alpha = 0.3, linestyle = '--', zorder = -1)

plt.xticks(fontsize = 21)
plt.yticks(fontsize = 21)
plt.ylabel("Time (s)", fontsize = 21)
plt.title("Duration of HR response associated with full-body movements", fontsize = 24)

d_sleep = np.array(d_sleep)
d_wake = np.array(d_wake)

# Ties are a problem for the Wilcoxon test
# d_sleep = d_sleep[d_sleep != 26]
# d_wake = d_wake[d_wake != 26]

from scipy import stats

w_stat, p_value = stats.wilcoxon(d_sleep, d_wake)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

W-statistic: 0.0, p-value: 0.011616044899262472


/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/Users/marcellosicbaldi/anaconda3/lib/python3.10/site-packages/scipy/stats/_morestats.py:4102: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


In [92]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/boxplot_duration_sleep_wake.png", dpi = 300, bbox_inches = 'tight')

## Biphasic response - All during sleep!!

In [18]:
# With trunk
bursts_HR_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)
bursts_lower_body = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"LL", "RL"}) | (bursts_HR_sleep["Limbs"] == {"LL", "RL", "T"}) | (bursts_HR_sleep["Limbs"] == {"LL"}) | (bursts_HR_sleep["Limbs"] == {"RL"})].reset_index(drop=True)

HR_response_mean_within_lower_body = bursts_lower_body.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_lower_body.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_lower_body.dropna().to_numpy().shape[0])

t = np.arange(-19, 50)
plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_lower_body.mean(), label = "lower body", color = 'blue')
plt.errorbar(t, HR_response_mean_within_lower_body.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

# No trunk, single limbs
bursts_lower_body = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"LL", "RL"}) | (bursts_HR_sleep["Limbs"] == {"LL"}) | (bursts_HR_sleep["Limbs"] == {"RL"})].reset_index(drop=True)

HR_response_mean_within_lower_body = bursts_lower_body.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_lower_body.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_lower_body.dropna().to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_lower_body.mean(), label = "lower body", color = 'blue')
plt.errorbar(t, HR_response_mean_within_lower_body.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

# No trunk, both legs together
bursts_lower_body = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"LL", "RL"}) | (bursts_HR_sleep["Limbs"] == {"LL", "RL", "T"})].reset_index(drop=True)

HR_response_mean_within_lower_body = bursts_lower_body.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_lower_body.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_lower_body.dropna().to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_lower_body.mean(), label = "lower body", color = 'blue')
plt.errorbar(t, HR_response_mean_within_lower_body.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

<ErrorbarContainer object of 3 artists>

In [19]:
# With trunk
bursts_upper_body = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"LW", "RW"}) | (bursts_HR_sleep["Limbs"] == {"LW", "RW", "T"}) | (bursts_HR_sleep["Limbs"] == {"LW"}) | (bursts_HR_sleep["Limbs"] == {"RW"})].reset_index(drop=True)

HR_response_mean_within_upper_body = bursts_upper_body.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_upper_body.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_upper_body.dropna().to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_upper_body.mean(), label = "upper body", color = 'blue')
plt.errorbar(t, HR_response_mean_within_upper_body.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

# No trunk
bursts_upper_body = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"LW", "RW"}) | (bursts_HR_sleep["Limbs"] == {"LW"}) | (bursts_HR_sleep["Limbs"] == {"RW"})].reset_index(drop=True)

HR_response_mean_within_upper_body = bursts_upper_body.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_upper_body.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_upper_body.dropna().to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_upper_body.mean(), label = "upper body", color = 'blue')
plt.errorbar(t, HR_response_mean_within_upper_body.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

# No trunk
bursts_upper_body = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"LW", "RW"}) | (bursts_HR_sleep["Limbs"] == {"LW", "RW", "T"})].reset_index(drop=True)

HR_response_mean_within_upper_body = bursts_upper_body.groupby("sub_ID")["HR_response_normalized"].mean()

sem = HR_response_mean_within_upper_body.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_upper_body.dropna().to_numpy().shape[0])

plt.figure(figsize=(15, 9))
plt.plot(t, HR_response_mean_within_upper_body.mean(), label = "upper body", color = 'blue')
plt.errorbar(t, HR_response_mean_within_upper_body.mean(), yerr = sem, fmt = '-o', color = 'blue', ecolor='blue', capsize=3)

<ErrorbarContainer object of 3 artists>

In [14]:
HR_response_mean_within_lower_body

sub_ID
098    [-0.23605905283985607, -0.9187165380516181, -0...
158    [0.378837924898634, 0.314964051011998, 0.56315...
279    [-1.0853836817395244, -0.861272781639321, -1.3...
547    [0.9313790080302085, 0.20124597195751953, -0.3...
633    [0.6670504502643098, -0.12037741092649697, -0....
815    [9.755388220052494, 5.15465818878665, 3.270111...
906    [-0.2815997938984817, 0.2668790212434177, 0.20...
958    [7.645181290452397, -5.6898003195477855, -1.68...
971    [-0.40077840198893866, -1.7121684421170376, 0....
Name: HR_response_normalized, dtype: object

In [57]:
bursts_HR_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)
bursts_lower_body = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"LL", "RL"}) | (bursts_HR_sleep["Limbs"] == {"LL", "RL", "T"}) | (bursts_HR_sleep["Limbs"] == {"LL"}) | (bursts_HR_sleep["Limbs"] == {"RL"}) | (bursts_HR_sleep["Limbs"] == {"LL", "T"}) | (bursts_HR_sleep["Limbs"] == {"RL", "T"})].reset_index(drop=True)
HR_response_mean_within_lower_body = bursts_lower_body.groupby("sub_ID")["HR_response_normalized"].mean()
sem_lower_body = HR_response_mean_within_lower_body.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_lower_body.dropna().to_numpy().shape[0])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17, 7), sharex=True, sharey=True)

t = np.arange(-19, 40)

# Top plot: lower body
ax1.errorbar(t, HR_response_mean_within_lower_body.mean(), yerr = sem_lower_body, fmt = '-o', capsize=3, elinewidth=2.1, label = "Lower-body\nmovement")
ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-2), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

plt.xlim(-12.5, 26.5)

ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
plt.ylim(-5,10)
bursts_upper_body = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"LW", "RW"}) | (bursts_HR_sleep["Limbs"] == {"LW", "RW", "T"}) | (bursts_HR_sleep["Limbs"] == {"LW"}) | (bursts_HR_sleep["Limbs"] == {"RW"}) | (bursts_HR_sleep["Limbs"] == {"LW", "T"}) | (bursts_HR_sleep["Limbs"] == {"RW", "T"})].reset_index(drop=True)
HR_response_mean_within_upper_body = bursts_upper_body.groupby("sub_ID")["HR_response_normalized"].mean()
sem_upper_body = HR_response_mean_within_upper_body.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_upper_body.dropna().to_numpy().shape[0])

# Bottom plot: upper body
ax2.errorbar(t, HR_response_mean_within_upper_body.mean(), yerr = sem_upper_body, fmt = '-o', capsize=3, elinewidth=2.1, color = sns.color_palette("Set1")[1], label = "Upper-body\nmovement")
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax2.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-2), xytext=(-60, plt.ylim()[1]-66),
                textcoords='offset points', ha='right', va='bottom', fontsize=19,
                bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
                arrowprops=dict(facecolor='black', shrink=0.05, width=2))

ax2.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax2.set_xlabel('Time (seconds)', fontsize = 21)
ax2.xaxis.set_tick_params(labelsize=20)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)

plt.tight_layout(w_pad=4)

In [66]:
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/paper/HR_response_upper_lower_body.png", dpi = 300, bbox_inches = 'tight')

## Ipsilateral

In [89]:
bursts_left.shape, bursts_right.shape

((4, 14), (138, 14))

In [90]:
bursts_HR_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)
bursts_left = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"LL", "LW"}) | (bursts_HR_sleep["Limbs"] == {"LL", "LW", "T"}) | (bursts_HR_sleep["Limbs"] == {"LL"}) | (bursts_HR_sleep["Limbs"] == {"LW"}) | (bursts_HR_sleep["Limbs"] == {"LL", "T"}) | (bursts_HR_sleep["Limbs"] == {"LW", "T"})].reset_index(drop=True)
HR_response_mean_within_left = bursts_left.groupby("sub_ID")["HR_response_normalized"].mean()
sem_left = HR_response_mean_within_left.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_left.dropna().to_numpy().shape[0])

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17, 7), sharex=True, sharey=True)

t = np.arange(-19, 50)

# Top plot: lower body
ax1.errorbar(t, HR_response_mean_within_left.mean(), yerr = sem_left, fmt = '-o', capsize=3, elinewidth=2.1, label = "Left body\nmovement")
ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-2), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

plt.xlim(-12.5, 26.5)

ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
plt.ylim(-4,7)

bursts_right = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"RL", "RW"}) | (bursts_HR_sleep["Limbs"] == {"RL", "RW", "T"}) | (bursts_HR_sleep["Limbs"] == {"RL"}) | (bursts_HR_sleep["Limbs"] == {"RW"}) | (bursts_HR_sleep["Limbs"] == {"RL", "T"}) | (bursts_HR_sleep["Limbs"] == {"RW", "T"})].reset_index(drop=True)
HR_response_mean_within_right = bursts_right.groupby("sub_ID")["HR_response_normalized"].mean()
sem_right = HR_response_mean_within_right.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_right.dropna().to_numpy().shape[0])

# Bottom plot: upper body
ax2.errorbar(t, HR_response_mean_within_right.mean(), yerr = sem_right, fmt = '-o', capsize=3, elinewidth=2.1, color = sns.color_palette("Set1")[1], label = "Right body\nmovement")
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax2.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax2.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-2), xytext=(-60, plt.ylim()[1]-66),
                textcoords='offset points', ha='right', va='bottom', fontsize=19,
                bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
                arrowprops=dict(facecolor='black', shrink=0.05, width=2))

ax2.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax2.set_xlabel('Time (seconds)', fontsize = 21)
ax2.xaxis.set_tick_params(labelsize=20)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)

plt.tight_layout(w_pad=4)

In [70]:
plt.savefig("/Users/marcellosicbaldi/Documents/GitHub/Movement-HR-Sleep/figures/paper/HR_response_left_right.png", dpi = 300, bbox_inches = 'tight')

## Controlateral -- too few!

In [78]:
bursts_HR_sleep = bursts_HR#[bursts_HR["SIB"] == 1].reset_index(drop=True)
bursts_contr = bursts_HR_sleep[(bursts_HR_sleep["Limbs"] == {"LL", "RW"}) | (bursts_HR_sleep["Limbs"] == {"LL", "RW", "T"}) | (bursts_HR_sleep["Limbs"] == {"RL", "LW"}) | (bursts_HR_sleep["Limbs"] == {"RL", "LW", "T"})].reset_index(drop=True)
HR_response_mean_within_contr = bursts_contr.groupby("sub_ID")["HR_response_normalized"].mean()
sem_contr = HR_response_mean_within_contr.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_contr.dropna().to_numpy().shape[0])

fig, (ax1) = plt.subplots(1, 1, figsize=(12, 7), sharex=True, sharey=True)

t = np.arange(-19, 50)

# Top plot: lower body
ax1.errorbar(t, HR_response_mean_within_contr.mean(), yerr = sem_contr, fmt = '-o', capsize=3, elinewidth=2.1, label = "Left body\nmovement")
ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax1.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax1.annotate('Movement\nonset', xy=(0, plt.ylim()[1]-2), xytext=(-60, plt.ylim()[1]-66),
             textcoords='offset points', ha='right', va='bottom', fontsize=19,
             bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
             arrowprops=dict(facecolor='black', shrink=0.05, width=2))

plt.xlim(-12.5, 26.5)

ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_ylabel('HR change', fontsize = 21)
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)

## 1 vs 2 vs 3 vs 4 vs 5 limbs

In [20]:
bursts_HR["n_limbs"] = bursts_HR["Limbs"].apply(lambda x: len(x))

In [50]:
plt.figure(figsize=(15, 9))
for sub in subjects:
    plt.plot(bursts_HR_3limbs.groupby("sub_ID")["HR_response_normalized"].mean().loc[sub])

plt.figure(figsize=(15, 9))
for sub in subjects:
    plt.plot(bursts_HR_4limbs.groupby("sub_ID")["HR_response_normalized"].mean().loc[sub])


In [24]:
bursts_HR_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)
bursts_HR_1limb = bursts_HR_sleep[bursts_HR_sleep["n_limbs"] == 1].reset_index(drop=True)
HR_response_mean_within_1limb = bursts_HR_1limb.groupby("sub_ID")["HR_response_normalized"].mean()
sem_1limb = HR_response_mean_within_1limb.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_1limb.dropna().to_numpy().shape[0])

bursts_HR_2limbs = bursts_HR_sleep[(bursts_HR_sleep["n_limbs"] >= 2) & (bursts_HR_sleep["n_limbs"] != 5)].reset_index(drop=True)
HR_response_mean_within_2limbs = bursts_HR_2limbs.groupby("sub_ID")["HR_response_normalized"].mean()
sem_2limbs = HR_response_mean_within_2limbs.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_2limbs.dropna().to_numpy().shape[0])

# bursts_HR_3limbs = bursts_HR_sleep[bursts_HR_sleep["n_limbs"] == 3].reset_index(drop=True)
# HR_response_mean_within_3limbs = bursts_HR_3limbs.groupby("sub_ID")["HR_response_normalized"].mean().drop("547")
# sem_3limbs = HR_response_mean_within_3limbs.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_3limbs.dropna().to_numpy().shape[0])

# bursts_HR_4limbs = bursts_HR_sleep[bursts_HR_sleep["n_limbs"] == 4].reset_index(drop=True)
# HR_response_mean_within_4limbs = bursts_HR_4limbs.groupby("sub_ID")["HR_response_normalized"].mean().drop("547")
# sem_4limbs = HR_response_mean_within_4limbs.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_4limbs.dropna().to_numpy().shape[0])

bursts_HR_5limbs = bursts_HR_sleep[bursts_HR_sleep["n_limbs"] == 5].reset_index(drop=True)
HR_response_mean_within_5limbs = bursts_HR_5limbs.groupby("sub_ID")["HR_response_normalized"].mean()
sem_5limbs = HR_response_mean_within_5limbs.dropna().to_numpy().std() / np.sqrt(HR_response_mean_within_5limbs.dropna().to_numpy().shape[0])

fig, (ax) = plt.subplots(3, 2, figsize=(12, 7), sharex=True, sharey=False)

t = np.arange(-19, 50)

# Top plot: 1 limb
ax[0, 0].errorbar(t, HR_response_mean_within_1limb.mean(), yerr = sem_1limb, fmt = '-o', capsize=3, elinewidth=2.1, label = "1 limb")
ax[0, 0].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[0, 0].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax[0, 0].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax[0, 0].set_ylabel('HR change', fontsize = 21)
ax[0, 0].set_label('')
ax[0, 0].xaxis.set_tick_params(labelsize=20)
ax[0, 0].yaxis.set_tick_params(labelsize=20)
ax[0, 0].set_xlabel('Time (seconds)', fontsize = 21)
ax[0, 0].set_xlim(-15, 40)
ax[0, 0].legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
ax[0, 0].set_title('1 limb', fontsize = 21)

# Top plot: >=2 limbs 
ax[0, 1].errorbar(t, HR_response_mean_within_2limbs.mean(), yerr = sem_2limbs, fmt = '-o', capsize=3, elinewidth=2.1, label = "2 limbs")
ax[0, 1].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[0, 1].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax[0, 1].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax[0, 1].set_label('')
ax[0, 1].xaxis.set_tick_params(labelsize=20)
ax[0, 1].yaxis.set_tick_params(labelsize=20)
ax[0, 1].set_xlabel('Time (seconds)', fontsize = 21)
ax[0, 1].set_xlim(-15, 40)
ax[0, 1].legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
ax[0, 1].set_title('2 limbs', fontsize = 21)

# # Top plot: 3 limbs
# ax[1, 0].errorbar(t, HR_response_mean_within_3limbs.mean(), yerr = sem_3limbs, fmt = '-o', capsize=3, elinewidth=2.1, label = "3 limbs")
# ax[1, 0].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
# ax[1, 0].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
# ax[1, 0].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
# ax[1, 0].set_ylabel('HR change', fontsize = 21)
# ax[1, 0].set_label('')
# ax[1, 0].xaxis.set_tick_params(labelsize=20)
# ax[1, 0].yaxis.set_tick_params(labelsize=20)
# ax[1, 0].set_xlabel('Time (seconds)', fontsize = 21)
# ax[1, 0].set_xlim(-15, 40)
# ax[1, 0].legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
# ax[1, 0].set_title('3 limbs', fontsize = 21)

# # Top plot: 4 limbs
# ax[1, 1].errorbar(t, HR_response_mean_within_4limbs.mean(), yerr = sem_4limbs, fmt = '-o', capsize=3, elinewidth=2.1, label = "4 limbs")
# ax[1, 1].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
# ax[1, 1].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
# ax[1, 1].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
# ax[1, 1].set_label('')
# ax[1, 1].xaxis.set_tick_params(labelsize=20)
# ax[1, 1].yaxis.set_tick_params(labelsize=20)
# ax[1, 1].set_xlabel('Time (seconds)', fontsize = 21)
# ax[1, 1].legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
# ax[1, 1].legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
# ax[1, 1].set_title('4 limbs', fontsize = 21)

# Top plot: 5 limbs
ax[2, 0].errorbar(t, HR_response_mean_within_5limbs.mean(), yerr = sem_5limbs, fmt = '-o', capsize=3, elinewidth=2.1, label = "5 limbs")
ax[2, 0].axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax[2, 0].axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
ax[2, 0].yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax[2, 0].set_ylabel('HR change', fontsize = 21)
ax[2, 0].set_label('')
ax[2, 0].xaxis.set_tick_params(labelsize=20)
ax[2, 0].yaxis.set_tick_params(labelsize=20)
ax[2, 0].set_xlabel('Time (seconds)', fontsize = 21)
ax[2, 0].set_xlim(-15, 40)
ax[2, 0].legend(frameon=True, fancybox=True, shadow=True, framealpha=1, fontsize=19)
ax[2, 0].set_title('5 limbs', fontsize = 21)

plt.tight_layout()

In [40]:
bursts_HR_4limbs.groupby("sub_ID")["AUC"].mean().plot(kind = "bar")

<Axes: xlabel='sub_ID'>

In [41]:
bursts_HR_3limbs.groupby("sub_ID")["AUC"].mean().plot(kind = "bar", color = "blue")

<Axes: xlabel='sub_ID'>

# Acc tertiles

In [50]:
bursts_HR_all_limbs = bursts_HR[bursts_HR["Limbs"] == set(("LL", "LW", "RL", "RW", "T"))].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33 = HR_response_percentiles["33.3%"]
AUC_66 = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] < AUC_33.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs.loc[(bursts_HR_all_limbs["sub_ID"] == sub) & (bursts_HR_all_limbs["AUC"] > AUC_66.loc[sub]), "HR_response_category"] = "high"
HR_response_SPT = bursts_HR_all_limbs.groupby(["sub_ID", "HR_response_category"])["ACC_response"].mean().unstack()
sem_high_SPT = HR_response_SPT["high"].to_numpy().std() / np.sqrt(HR_response_SPT["high"].to_numpy().shape[0])
sem_medium_SPT = HR_response_SPT["medium"].to_numpy().std() / np.sqrt(HR_response_SPT["medium"].to_numpy().shape[0])
sem_low_SPT = HR_response_SPT["low"].to_numpy().std() / np.sqrt(HR_response_SPT["low"].to_numpy().shape[0])

# Sleep
bursts_HR_all_limbs_sleep = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 1].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_sleep.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_sleep = HR_response_percentiles["33.3%"]
AUC_66_sleep = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_sleep["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] < AUC_33_sleep.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_sleep.loc[(bursts_HR_all_limbs_sleep["sub_ID"] == sub) & (bursts_HR_all_limbs_sleep["AUC"] > AUC_66_sleep.loc[sub]), "HR_response_category"] = "high"
HR_response_SLEEP_acc = bursts_HR_all_limbs_sleep.groupby(["sub_ID", "HR_response_category"])["ACC_response"].mean().unstack()
sem_high_SLEEP_acc = HR_response_SLEEP_acc["high"].to_numpy().std() / np.sqrt(HR_response_SLEEP_acc["high"].to_numpy().shape[0])
sem_medium_SLEEP_acc = HR_response_SLEEP_acc["medium"].to_numpy().std() / np.sqrt(HR_response_SLEEP_acc["medium"].to_numpy().shape[0])
sem_low_SLEEP_acc = HR_response_SLEEP_acc["low"].to_numpy().std() / np.sqrt(HR_response_SLEEP_acc["low"].to_numpy().shape[0])

# Wake
bursts_HR_all_limbs_wake = bursts_HR_all_limbs[bursts_HR_all_limbs["SIB"] == 0].reset_index(drop=True)
# divide in low, med and high HR response based on the 1/3 and 2/3 percentiles
HR_response_percentiles = bursts_HR_all_limbs_wake.groupby("sub_ID")["AUC"].describe(percentiles=[1/3, 2/3])
AUC_33_wake = HR_response_percentiles["33.3%"]
AUC_66_wake = HR_response_percentiles["66.7%"]
bursts_HR_all_limbs_wake["HR_response_category"] = "medium"
for i, sub in enumerate(subjects):
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] < AUC_33_wake.loc[sub]), "HR_response_category"] = "low"
    bursts_HR_all_limbs_wake.loc[(bursts_HR_all_limbs_wake["sub_ID"] == sub) & (bursts_HR_all_limbs_wake["AUC"] > AUC_66_wake.loc[sub]), "HR_response_category"] = "high"
HR_response_WAKE = bursts_HR_all_limbs_wake.groupby(["sub_ID", "HR_response_category"])["ACC_response"].mean().unstack()
sem_high_WAKE = HR_response_WAKE["high"].to_numpy().std() / np.sqrt(HR_response_WAKE["high"].to_numpy().shape[0])
sem_medium_WAKE = HR_response_WAKE["medium"].to_numpy().std() / np.sqrt(HR_response_WAKE["medium"].to_numpy().shape[0])
sem_low_WAKE = HR_response_WAKE["low"].to_numpy().std() / np.sqrt(HR_response_WAKE["low"].to_numpy().shape[0])

In [53]:
t = np.arange(-19, 50)
f, ax = plt.subplots(1, 1, figsize=(15, 6))

plt.plot(t, HR_response_SLEEP_acc["low"].mean(), label = "Small Movement", color = 'blue', linewidth = 2.1)
plt.errorbar(t, HR_response_SLEEP_acc["low"].mean(), yerr = sem_low_SLEEP_acc, fmt = '-o', color = 'blue', ecolor='blue', capsize=3, elinewidth=1.5)
# shade the region below the curve
#plt.fill_between(t, HR_response_SLEEP_acc["low"].mean() - sem_low_SLEEP, HR_response_SLEEP_acc["low"].mean() + sem_low_SLEEP, color = 'blue')

plt.plot(t, HR_response_SLEEP_acc["medium"].mean(), label = "Medium Movement", color = 'green', linewidth = 2.1)
plt.errorbar(t, HR_response_SLEEP_acc["medium"].mean(), yerr = sem_medium_SLEEP_acc, fmt = '-o', color = 'green', ecolor='green', capsize=3, elinewidth=1.5)

plt.plot(t, HR_response_SLEEP_acc["high"].mean(), label = "Large Movement", color = 'red', linewidth = 2.1)
plt.errorbar(t, HR_response_SLEEP_acc["high"].mean(), yerr = sem_high_SLEEP_acc, fmt = '-o', color = 'red', ecolor='red', capsize=3, elinewidth=1.5)

ax.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)
# ax.annotate('Movement onset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
#              textcoords='offset points', ha='right', va='bottom', fontsize=19,
#              bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
#              arrowprops=dict(facecolor='black', shrink=0.05, width=2))

plt.xlim(-15, 30)
plt.title('Movement bursts - Sleep', fontsize = 21)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax.set_xlabel('Time (seconds)', fontsize = 21)
ax.set_ylabel('Envelopes difference change (%)', fontsize = 21)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax.set_label('')
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)

plt.legend(frameon = True, fancybox = True, shadow = True, fontsize = 18, loc = "upper right")

In [54]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/acc_tertiles.png", dpi = 300, bbox_inches = 'tight')

# EDA - General sleep parameters

In [ ]:
plt.figure(figsize=(12, 6))
sns.regplot(data = bursts_HR_all_limbs_sleep, x = "p2p", y = "AUC", color = 'black', scatter_kws = {"s": 50})

<Axes: xlabel='p2p', ylabel='AUC'>

In [288]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815", "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

tst = {sub: 0 for sub in subjects} # total sleep time
waso = {sub: 0 for sub in subjects} # wake after sleep onset
se = {sub: 0 for sub in subjects} # sleep efficiency
spt_dur = {sub: 0 for sub in subjects} # sleep period time (from diary)
n_awakenings = {sub: 0 for sub in subjects}

SIB = {sub: 0 for sub in subjects}

bursts_df = pd.DataFrame()

for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_FINAL_envInterp_p2p.pkl', 'rb') as f:
        bursts = pickle.load(f)

    df_merged_intervals = characterize_bursts(bursts)
    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('5 min')
    spt_dur[sub] = (diary_SPT[sub][1] - diary_SPT[sub][0]).total_seconds() / 60

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    # Take df_merged_intervals between spt_start and spt_end
    df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
    SIB[sub]["sib.duration"] = SIB[sub]["sib.end.time"] - SIB[sub]["sib.onset.time"]
    SIB[sub]["sub_ID"] = sub

    tst[sub] = (SIB[sub]["sib.duration"].sum()).total_seconds() / 60
    waso[sub] = (SIB[sub]["awake.duration"].sum()).total_seconds() / 60
    se[sub] = 100 * (tst[sub] / spt_dur[sub])
    n_awakenings[sub] = SIB[sub].shape[0]

    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    df_merged_intervals["sub_ID"] = sub

    start_sleep = diary_SPT[sub][0]
    end_sleep = diary_SPT[sub][1]

    df_merged_intervals = df_merged_intervals.loc[(df_merged_intervals["Start"] >= start_sleep) & (df_merged_intervals["End"] <= end_sleep)]

    bursts_df = pd.concat([bursts_df, df_merged_intervals])

sleep_parameters = pd.DataFrame([spt_dur, tst, waso, se, n_awakenings]).T
sleep_parameters.columns = ["spt_duration", "tst", "waso", "se", "n_awakenings"]

In [291]:
sleep_parameters.describe().drop("count")#.to_excel("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/tables/sleep_parameters.xlsx")

,spt_duration,tst,waso,se,n_awakenings
mean,451.916667,401.166667,45.631944,88.776035,16.833333
std,34.618593,40.918310,23.035312,6.209778,5.356955
min,391.000000,319.500000,4.916667,77.434457,5.000000
25%,432.750000,385.750000,35.125000,84.762429,14.750000
50%,462.000000,401.250000,40.750000,90.583966,17.500000
75%,480.000000,428.562500,62.187500,93.987475,19.500000
max,495.000000,453.250000,92.083333,97.730740,24.000000


## AUC tertiles

In [97]:
HR_burst_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)
plt.figure(figsize=(15, 9))
sns.histplot(data = HR_burst_sleep., x = "AUC", hue = "sub_ID", kde = True, bins = 100, palette = "Set1")

<Axes: xlabel='AUC', ylabel='Count'>

# Table LMM SMM

Inspired by Ibrahim2023

In [44]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815", "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

tst = {sub: 0 for sub in subjects}

SIB = {sub: 0 for sub in subjects}

bursts_df = pd.DataFrame()

for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_FINAL_envInterp_p2p.pkl', 'rb') as f:
        bursts = pickle.load(f)

    df_merged_intervals = characterize_bursts(bursts)
    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    # Take df_merged_intervals between spt_start and spt_end
    df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
    SIB[sub]["sib.duration"] = SIB[sub]["sib.end.time"] - SIB[sub]["sib.onset.time"]
    SIB[sub]["sub_ID"] = sub

    tst[sub] = (SIB[sub]["sib.duration"].sum()).total_seconds() / 3600

    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    df_merged_intervals["sub_ID"] = sub

    start_sleep = diary_SPT[sub][0]
    end_sleep = diary_SPT[sub][1]

    df_merged_intervals = df_merged_intervals.loc[(df_merged_intervals["Start"] >= start_sleep) & (df_merged_intervals["End"] <= end_sleep)]

    bursts_df = pd.concat([bursts_df, df_merged_intervals])

In [102]:
bursts_df_sleep = bursts_df[bursts_df["SIB"] == 1].reset_index(drop=True)
bursts_df_sleep["Duration"] = bursts_df_sleep["End"] - bursts_df_sleep["Start"]
bursts_df_sleep["Duration"] = bursts_df_sleep["Duration"].apply(lambda x: x.total_seconds())
bursts_df_sleep["n_limbs"] = bursts_df_sleep["Limbs"].apply(lambda x: len(x))
bursts_df_LMM = bursts_df_sleep[(bursts_df_sleep["n_limbs"] >= 2)].reset_index(drop=True)
bursts_df_fullBody = bursts_df_sleep[(bursts_df_sleep["n_limbs"] == 5)].reset_index(drop=True)
bursts_df_upper = bursts_df_sleep[(bursts_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_HR_sleep["Limbs"] == {"LL", "RL", "T"})]
bursts_df_lower = bursts_df_sleep[(bursts_df_sleep["Limbs"] == {"LW", "RW"}) | (bursts_HR_sleep["Limbs"] == {"LW", "RW", "T"})]
bursts_df_LMM_others = bursts_df_sleep[(bursts_df_sleep["n_limbs"] >= 2) & (bursts_df_sleep["n_limbs"] != 5) & (bursts_df_sleep["Limbs"] != {"LL", "RL"}) & (bursts_df_sleep["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_sleep["Limbs"] != {"LW", "RW"}) & (bursts_df_sleep["Limbs"] != {"LW", "RW", "T"})].reset_index(drop=True)
bursts_df_SMM = bursts_df_sleep[(bursts_df_sleep["n_limbs"] == 1)].reset_index(drop=True)

bursts_df_LMM["Duration"].describe()
bursts_df_SMM["Duration"].describe()

duration_LMM_50 = bursts_df_LMM.groupby("sub_ID")["Duration"].mean()
duration_LMM_25 = bursts_df_LMM.groupby("sub_ID")["Duration"].quantile(0.25).mean()
duration_LMM_75 = bursts_df_LMM.groupby("sub_ID")["Duration"].quantile(0.75).mean()

duration_fullBody_50 = bursts_df_fullBody.groupby("sub_ID")["Duration"].mean()
duration_fullBody_25 = bursts_df_fullBody.groupby("sub_ID")["Duration"].quantile(0.25).mean()
duration_fullBody_75 = bursts_df_fullBody.groupby("sub_ID")["Duration"].quantile(0.75).mean()

duration_upper_50 = bursts_df_upper.groupby("sub_ID")["Duration"].mean()
duration_upper_25 = bursts_df_upper.groupby("sub_ID")["Duration"].quantile(0.25).mean()
duration_upper_75 = bursts_df_upper.groupby("sub_ID")["Duration"].quantile(0.75).mean()

duration_lower_50 = bursts_df_lower.groupby("sub_ID")["Duration"].mean()
duration_lower_25 = bursts_df_lower.groupby("sub_ID")["Duration"].quantile(0.25).mean()
duration_lower_75 = bursts_df_lower.groupby("sub_ID")["Duration"].quantile(0.75).mean()

duration_other_50 = bursts_df_LMM_others.groupby("sub_ID")["Duration"].mean()
duration_other_25 = bursts_df_LMM_others.groupby("sub_ID")["Duration"].quantile(0.25).mean()
duration_other_75 = bursts_df_LMM_others.groupby("sub_ID")["Duration"].quantile(0.75).mean()

duration_SMM_50 = bursts_df_SMM.groupby("sub_ID")["Duration"].mean()
duration_SMM_25 = bursts_df_SMM.groupby("sub_ID")["Duration"].quantile(0.25).mean()
duration_SMM_75 = bursts_df_SMM.groupby("sub_ID")["Duration"].quantile(0.75).mean()

p2p_LMM_50 = bursts_df_LMM.groupby("sub_ID")["p2p"].mean()
p2p_LMM_25 = bursts_df_LMM.groupby("sub_ID")["p2p"].quantile(0.25).mean()
p2p_LMM_75 = bursts_df_LMM.groupby("sub_ID")["p2p"].quantile(0.75).mean()

p2p_fullBody_50 = bursts_df_fullBody.groupby("sub_ID")["p2p"].mean()
p2p_fullBody_25 = bursts_df_fullBody.groupby("sub_ID")["p2p"].quantile(0.25).mean()
p2p_fullBody_75 = bursts_df_fullBody.groupby("sub_ID")["p2p"].quantile(0.75).mean()

p2p_upper_50 = bursts_df_upper.groupby("sub_ID")["p2p"].mean()
p2p_upper_25 = bursts_df_upper.groupby("sub_ID")["p2p"].quantile(0.25).mean()
p2p_upper_75 = bursts_df_upper.groupby("sub_ID")["p2p"].quantile(0.75).mean()

p2p_lower_50 = bursts_df_lower.groupby("sub_ID")["p2p"].mean()
p2p_lower_25 = bursts_df_lower.groupby("sub_ID")["p2p"].quantile(0.25).mean()
p2p_lower_75 = bursts_df_lower.groupby("sub_ID")["p2p"].quantile(0.75).mean()

p2p_other_50 = bursts_df_LMM_others.groupby("sub_ID")["p2p"].mean()
p2p_other_25 = bursts_df_LMM_others.groupby("sub_ID")["p2p"].quantile(0.25).mean()
p2p_other_75 = bursts_df_LMM_others.groupby("sub_ID")["p2p"].quantile(0.75).mean()

p2p_SMM_50 = bursts_df_SMM.groupby("sub_ID")["p2p"].mean()
p2p_SMM_25 = bursts_df_SMM.groupby("sub_ID")["p2p"].quantile(0.25).mean()
p2p_SMM_75 = bursts_df_SMM.groupby("sub_ID")["p2p"].quantile(0.75).mean()

AUC_LMM_50 = bursts_df_LMM.groupby("sub_ID")["AUC"].mean()
AUC_LMM_25 = bursts_df_LMM.groupby("sub_ID")["AUC"].quantile(0.25).mean()
AUC_LMM_75 = bursts_df_LMM.groupby("sub_ID")["AUC"].quantile(0.75).mean()

AUC_fullBody_50 = bursts_df_fullBody.groupby("sub_ID")["AUC"].mean()
AUC_fullBody_25 = bursts_df_fullBody.groupby("sub_ID")["AUC"].quantile(0.25).mean()
AUC_fullBody_75 = bursts_df_fullBody.groupby("sub_ID")["AUC"].quantile(0.75).mean()

AUC_upper_50 = bursts_df_upper.groupby("sub_ID")["AUC"].mean()
AUC_upper_25 = bursts_df_upper.groupby("sub_ID")["AUC"].quantile(0.25).mean()
AUC_upper_75 = bursts_df_upper.groupby("sub_ID")["AUC"].quantile(0.75).mean()

AUC_lower_50 = bursts_df_lower.groupby("sub_ID")["AUC"].mean()
AUC_lower_25 = bursts_df_lower.groupby("sub_ID")["AUC"].quantile(0.25).mean()
AUC_lower_75 = bursts_df_lower.groupby("sub_ID")["AUC"].quantile(0.75).mean()

AUC_other_50 = bursts_df_LMM_others.groupby("sub_ID")["AUC"].mean()
AUC_other_25 = bursts_df_LMM_others.groupby("sub_ID")["AUC"].quantile(0.25).mean()
AUC_other_75 = bursts_df_LMM_others.groupby("sub_ID")["AUC"].quantile(0.75).mean()

AUC_SMM_50 = bursts_df_SMM.groupby("sub_ID")["AUC"].mean()
AUC_SMM_25 = bursts_df_SMM.groupby("sub_ID")["AUC"].quantile(0.25).mean()
AUC_SMM_75 = bursts_df_SMM.groupby("sub_ID")["AUC"].quantile(0.75).mean()

tst_df = pd.DataFrame(tst, index = ["TST"]).T.sort_index()

index_LMM = bursts_df_LMM.groupby("sub_ID")["Start"].count() / tst_df["TST"]
index_fullBody = bursts_df_fullBody.groupby("sub_ID")["Start"].count() / tst_df["TST"]
index_upper = bursts_df_upper.groupby("sub_ID")["Start"].count() / tst_df["TST"]
index_lower = bursts_df_lower.groupby("sub_ID")["Start"].count() / tst_df["TST"]
index_other = bursts_df_LMM_others.groupby("sub_ID")["Start"].count() / tst_df["TST"]
index_SMM = bursts_df_SMM.groupby("sub_ID")["Start"].count() / tst_df["TST"]

index_LMM_50 = index_LMM.mean()
index_LMM_25 = index_LMM.quantile(0.25).mean()
index_LMM_75 = index_LMM.quantile(0.75).mean()

index_fullBody_50 = index_fullBody.mean()
index_fullBody_25 = index_fullBody.quantile(0.25).mean()
index_fullBody_75 = index_fullBody.quantile(0.75).mean()

index_upper_50 = index_upper.mean()
index_upper_25 = index_upper.quantile(0.25).mean()
index_upper_75 = index_upper.quantile(0.75).mean()

index_lower_50 = index_lower.mean()
index_lower_25 = index_lower.quantile(0.25).mean()
index_lower_75 = index_lower.quantile(0.75).mean()

index_other_50 = index_other.mean()
index_other_25 = index_other.quantile(0.25).mean()
index_other_75 = index_other.quantile(0.75).mean()

index_SMM_50 = index_SMM.mean()
index_SMM_25 = index_SMM.quantile(0.25).mean()
index_SMM_75 = index_SMM.quantile(0.75).mean()

# Put all of the data in a single dataframe

data = {
    "Duration": [duration_LMM_50.mean(), duration_fullBody_50.mean(), duration_upper_50.mean(), duration_lower_50.mean(), duration_other_50.mean(), duration_SMM_50.mean()],
    "Duration_25": [duration_LMM_25, duration_fullBody_25, duration_upper_25, duration_lower_25, duration_other_25, duration_SMM_25],
    "Duration_75": [duration_LMM_75, duration_fullBody_75, duration_upper_75, duration_lower_75, duration_other_75, duration_SMM_75],
    "p2p": [p2p_LMM_50.mean(), p2p_fullBody_50.mean(), p2p_upper_50.mean(), p2p_lower_50.mean(), p2p_other_50.mean(), p2p_SMM_50.mean()],
    "p2p_25": [p2p_LMM_25, p2p_fullBody_25, p2p_upper_25, p2p_lower_25, p2p_other_25, p2p_SMM_25],
    "p2p_75": [p2p_LMM_75, p2p_fullBody_75, p2p_upper_75, p2p_lower_75, p2p_other_75, p2p_SMM_75],
    "AUC": [AUC_LMM_50.mean(), AUC_fullBody_50.mean(), AUC_upper_50.mean(), AUC_lower_50.mean(), AUC_other_50.mean(), AUC_SMM_50.mean()],
    "AUC_25": [AUC_LMM_25, AUC_fullBody_25, AUC_upper_25, AUC_lower_25, AUC_other_25, AUC_SMM_25],
    "AUC_75": [AUC_LMM_75, AUC_fullBody_75, AUC_upper_75, AUC_lower_75, AUC_other_75, AUC_SMM_75],
    "Index": [index_LMM_50, index_fullBody_50, index_upper_50, index_lower_50, index_other_50, index_SMM_50],
    "Index_25": [index_LMM_25, index_fullBody_25, index_upper_25, index_lower_25, index_other_25, index_SMM_25],
        "Index_75": [index_LMM_75, index_fullBody_75, index_upper_75, index_lower_75, index_other_75, index_SMM_75]
}

df = pd.DataFrame(data, index = ["LMM", "Full Body", "Upper Limbs", "Lower Limbs", "Other Limbs", "SMM"])

#df.round(2).to_excel("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/tables/table1.xlsx")
df

,Duration,Duration_25,Duration_75,p2p,p2p_25,p2p_75,AUC,AUC_25,AUC_75,Index,Index_25,Index_75
LMM,6.615276,3.052138,8.141433,634.037638,121.086123,746.070652,190795.448862,18348.368632,181179.642966,13.377803,10.220291,16.224240
Full Body,10.296962,5.648363,12.402156,1214.984487,458.091229,1668.579219,400121.527796,103439.367030,522127.886597,5.428227,3.791913,5.806579
Upper Limbs,3.428389,1.949274,4.278076,110.790790,52.346389,125.064653,20131.131198,6283.789016,22106.758327,2.155727,1.262118,2.233489
Lower Limbs,3.051856,2.149417,3.769241,206.982888,100.001666,280.586180,36774.655978,14924.002531,54271.457996,0.705702,0.234254,0.989097
Other Limbs,4.939939,2.608696,6.390312,225.138434,96.959697,264.022595,49160.997783,13647.500232,63010.382609,4.146974,2.436501,4.607952
SMM,1.396044,0.536250,1.859791,36.659786,20.708005,39.132783,4810.767487,1048.097530,5648.333718,10.251923,7.647735,10.704056


# Table OP

Spatial distribution of movements was subdivided into two categories: focal movements and large muscle group movements (LMM). Focal movements were defined as localized to one body part [7], while LMM as temporally overlapping bursts in any combination of at least two body parts [8]. LMM were further categorized into: 
- Full-body movements: bursts involving all five sensors.
- Upper body movements: bursts involving both wrists or both wrists and trunk.
- Lower body movements: bursts involving both ankles or both ankles and trunk.
- Mixed movements: bursts involving both upper and lower extremities and different from full-body movements.

Movement laterality was described as unilateral if movements involved only one side of the body or bilateral if both sides were involved [7].
Finally, posture changes were identified as shifts of a least 30° in the orientation of the accelerometer placed on the lower back [9]. 


In [18]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815", "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

tst = {sub: 0 for sub in subjects}
twt = {sub: 0 for sub in subjects}

SIB = {sub: 0 for sub in subjects}

bursts_df = pd.DataFrame()

for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_FINAL_envInterp_p2p.pkl', 'rb') as f:
        bursts = pickle.load(f)

    df_merged_intervals = characterize_bursts(bursts)
    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    # Take df_merged_intervals between spt_start and spt_end
    df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
    SIB[sub]["sib.duration"] = SIB[sub]["sib.end.time"] - SIB[sub]["sib.onset.time"]
    SIB[sub]["sub_ID"] = sub

    tst[sub] = (SIB[sub]["sib.duration"].sum()).total_seconds() / 3600
    twt[sub] = (SIB[sub]["awake.duration"].sum()).total_seconds() / 3600

    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    df_merged_intervals["sub_ID"] = sub

    start_sleep = diary_SPT[sub][0]
    end_sleep = diary_SPT[sub][1]

    df_merged_intervals = df_merged_intervals.loc[(df_merged_intervals["Start"] >= start_sleep) & (df_merged_intervals["End"] <= end_sleep)]

    bursts_df = pd.concat([bursts_df, df_merged_intervals])
    
tst_df = pd.DataFrame(tst, index = ["TST"]).T.sort_index()
twt_df = pd.DataFrame(twt, index = ["TWT"]).T.sort_index()

In [7]:
bursts_df_sleep.head()

,Start,End,AUC,p2p,PC,transition,Limbs,SIB,sub_ID,Duration,n_limbs,Category,Laterality
0,2024-02-28 23:00:01.360290051,2024-02-28 23:00:02.460289955,4478.029974,59.446242,0.0,None,"{LL, RL, T}",1,158,1.099999,3,Lower Body,Bilateral
1,2024-02-28 23:00:30.130290031,2024-02-28 23:00:32.030289888,5565.732278,48.247713,0.0,None,{LL},1,158,1.899999,1,Focal,Unilateral
2,2024-02-28 23:00:43.494869947,2024-02-28 23:00:43.874870062,593.812924,16.318746,0.0,None,{T},1,158,0.380000,1,Focal,None
3,2024-02-28 23:02:28.765630007,2024-02-28 23:02:31.105629921,6965.555964,44.916305,0.0,None,"{LW, T}",1,158,2.339999,2,Upper Body,Unilateral
4,2024-02-28 23:03:30.215630054,2024-02-28 23:03:32.105629921,6534.732075,50.164798,0.0,None,"{LW, T}",1,158,1.889999,2,Upper Body,Unilateral


In [19]:
bursts_df_sleep = bursts_df[bursts_df["SIB"] == 1].reset_index(drop=True)
bursts_df_sleep["Duration"] = bursts_df_sleep["End"] - bursts_df_sleep["Start"]
bursts_df_sleep["Duration"] = bursts_df_sleep["Duration"].apply(lambda x: x.total_seconds())
bursts_df_sleep["n_limbs"] = bursts_df_sleep["Limbs"].apply(lambda x: len(x))
bursts_df_sleep["Category"] = "placeholder"
bursts_df_sleep["Laterality"] = "Bilateral"
bursts_df_sleep.loc[bursts_df_sleep["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_df_sleep["Limbs"] == {"LL", "T"}) | (bursts_df_sleep["Limbs"] == {"RL", "T"}) | (bursts_df_sleep["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LW", "RW"}) | (bursts_df_sleep["Limbs"] == {"LW", "T"}) | (bursts_df_sleep["Limbs"] == {"RW", "T"}) | (bursts_df_sleep["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df_sleep.loc[(bursts_df_sleep["n_limbs"] >= 2) & (bursts_df_sleep["n_limbs"] != 5) & (bursts_df_sleep["Limbs"] != {"LL", "RL"}) & (bursts_df_sleep["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_sleep["Limbs"] != {"LW", "RW"}) & (bursts_df_sleep["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_sleep["Limbs"] != {"LL", "T"}) & (bursts_df_sleep["Limbs"] != {"RL", "T"}) & (bursts_df_sleep["Limbs"] != {"LW", "T"}) & (bursts_df_sleep["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df_sleep.loc[bursts_df_sleep["n_limbs"] == 1, "Category"] = "Focal"
bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "LW"}) | (bursts_df_sleep["Limbs"] == {"RL", "RW"}) | (bursts_df_sleep["Limbs"] == {"LL", "LW", "T"}) | (bursts_df_sleep["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "T"}) | (bursts_df_sleep["Limbs"] == {"LW", "T"}) | (bursts_df_sleep["Limbs"] == {"RL", "T"}) | (bursts_df_sleep["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_df_sleep["Limbs"] == {"LW"}) | (bursts_df_sleep["Limbs"] == {"RW"}) | (bursts_df_sleep["Limbs"] == {"LL"}) | (bursts_df_sleep["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_df_sleep.loc[bursts_df_sleep["Limbs"] == {"T"}, "Laterality"] = "None"

bursts_df_wake = bursts_df[bursts_df["SIB"] == 0].reset_index(drop=True)
bursts_df_wake["Duration"] = bursts_df_wake["End"] - bursts_df_wake["Start"]
bursts_df_wake["Duration"] = bursts_df_wake["Duration"].apply(lambda x: x.total_seconds())
bursts_df_wake["n_limbs"] = bursts_df_wake["Limbs"].apply(lambda x: len(x))
bursts_df_wake["Category"] = "placeholder"
bursts_df_wake["Laterality"] = "Bilateral"
bursts_df_wake.loc[bursts_df_wake["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "RL"}) | (bursts_df_wake["Limbs"] == {"LL", "T"}) | (bursts_df_wake["Limbs"] == {"RL", "T"}) | (bursts_df_wake["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LW", "RW"}) | (bursts_df_wake["Limbs"] == {"LW", "T"}) | (bursts_df_wake["Limbs"] == {"RW", "T"}) | (bursts_df_wake["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df_wake.loc[(bursts_df_wake["n_limbs"] >= 2) & (bursts_df_wake["n_limbs"] != 5) & (bursts_df_wake["Limbs"] != {"LL", "RL"}) & (bursts_df_wake["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_wake["Limbs"] != {"LW", "RW"}) & (bursts_df_wake["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_wake["Limbs"] != {"LL", "T"}) & (bursts_df_wake["Limbs"] != {"RL", "T"}) & (bursts_df_wake["Limbs"] != {"LW", "T"}) & (bursts_df_wake["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df_wake.loc[bursts_df_wake["n_limbs"] == 1, "Category"] = "Focal"
bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "LW"}) | (bursts_df_wake["Limbs"] == {"RL", "RW"}) | (bursts_df_wake["Limbs"] == {"LL", "LW", "T"}) | (bursts_df_wake["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "T"}) | (bursts_df_wake["Limbs"] == {"LW", "T"}) | (bursts_df_wake["Limbs"] == {"RL", "T"}) | (bursts_df_wake["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "RL"}) | (bursts_df_wake["Limbs"] == {"LW"}) | (bursts_df_wake["Limbs"] == {"RW"}) | (bursts_df_wake["Limbs"] == {"LL"}) | (bursts_df_wake["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_df_wake.loc[bursts_df_wake["Limbs"] == {"T"}, "Laterality"] = "None"

In [9]:
bursts_df.shape, bursts_df_sleep.shape + bursts_df_wake.shape

((2466, 9), (1698, 13, 768, 13))

In [21]:
movement_index = bursts_df_wake.groupby("sub_ID")["Duration"].count() / twt_df["TWT"]
movement_index.describe(percentiles=[0.25, 0.75, 0.9])

count     12.000000
mean      90.359088
std       21.429939
min       59.405941
25%       81.131145
50%       85.293105
75%       97.282027
90%      121.213908
max      130.370370
dtype: float64

In [22]:
duration = bursts_df_sleep.groupby("sub_ID")["Duration"].mean()
duration.describe(percentiles=[0.25, 0.75, 0.9])

duration = bursts_df_wake.groupby("sub_ID")["Duration"].mean()
duration.describe(percentiles=[0.25, 0.75, 0.9])

count    12.000000
mean     16.882710
std       6.945090
min       9.807177
25%      11.510286
50%      14.381433
75%      21.239613
90%      23.207790
max      33.189712
Name: Duration, dtype: float64

In [12]:
#print((bursts_df_wake.groupby(["Category", "sub_ID"])["Duration"].count().unstack().T.div(bursts_df_wake.groupby(["sub_ID"])["Duration"].count(), axis = 0)*100)[["Full Body"]].describe(percentiles=[0.25, 0.75, 0.9]))
print((bursts_df_wake.groupby(["Category", "sub_ID"])["Duration"].count().unstack().T.div(bursts_df_wake.groupby(["sub_ID"])["Duration"].count(), axis = 0)*100)["Focal"].describe(percentiles=[0.25, 0.75, 0.9]))
#(bursts_df_wake.groupby(["Category", "sub_ID"])["Duration"].count().unstack().T.div(bursts_df_wake.groupby(["sub_ID"])["Duration"].count(), axis = 0)*100)["Mixed"].describe(percentiles=[0.25, 0.75, 0.9])

count    12.000000
mean     15.704592
std       5.448969
min       5.319149
25%      12.782609
50%      14.753042
75%      19.423077
90%      21.411765
max      26.250000
Name: Focal, dtype: float64


Laterality

In [296]:
print(((bursts_df_wake.groupby(["sub_ID", "Laterality"])["Laterality"].count().unstack()["Unilateral"] / bursts_df_wake.groupby(["sub_ID"])["Duration"].count())*100).describe())
print(((bursts_df_wake.groupby(["sub_ID", "Laterality"])["Laterality"].count().unstack()["Bilateral"] / bursts_df_wake.groupby(["sub_ID"])["Duration"].count())*100).describe())

count    12.000000
mean     19.349721
std       7.139817
min       8.510638
25%      15.623188
50%      17.752326
75%      22.426471
max      33.750000
dtype: float64
count    12.000000
mean     79.873664
std       7.136044
min      66.250000
25%      75.622172
50%      81.279070
75%      84.376812
max      91.489362
dtype: float64


Posture

In [ ]:
bursts_df_sleep["PC"].replace(0, np.nan, inplace = True)
bursts_df_sleep["PC"] = bursts_df_sleep["PC"].apply(lambda x: 1 if x < 100 else np.nan)

bursts_df_wake["PC"].replace(0, np.nan, inplace = True)
bursts_df_wake["PC"] = bursts_df_wake["PC"].apply(lambda x: 1 if x < 100 else np.nan)

In [278]:
# Percentage of full body movements that are posture changes in sleep
n_tot_fullBody_sleep = bursts_df_sleep[bursts_df_sleep["Category"] == "Full Body"].shape[0]
(bursts_df_sleep.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"].sum() / n_tot_fullBody_sleep)*100

#median and IQR
((bursts_df_sleep.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"])).describe(percentiles=[0.25, 0.75, 0.9])

count    12.000000
mean      3.166667
std       3.010084
min       0.000000
25%       1.000000
50%       2.500000
75%       5.000000
90%       7.700000
max       9.000000
Name: Full Body, dtype: float64

In [279]:
# Percentage of full body movements that are posture changes in wake
n_tot_fullBody_wake = bursts_df_wake[bursts_df_wake["Category"] == "Full Body"].shape[0]
(bursts_df_wake.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"].sum() / n_tot_fullBody_wake)*100

#median and IQR
((bursts_df_wake.groupby(["sub_ID", "Category"])["PC"].count().unstack()["Full Body"])).describe(percentiles=[0.25, 0.75, 0.9])

count    12.000000
mean     12.000000
std       4.954337
min       6.000000
25%       9.000000
50%      11.000000
75%      14.000000
90%      19.400000
max      22.000000
Name: Full Body, dtype: float64

Makes sense that almost all posture changes are during wake since vanhees2015 is based on orientation changes of LW

Total number - within subjects

In [168]:
bursts_df_sleep.groupby(["Category", "sub_ID"])["Duration"].count().unstack().T.plot(kind = "bar", figsize = (15, 9), stacked = True)

<Axes: xlabel='sub_ID'>

Percentage

In [49]:
# same but expressed as percentage and write the percentage on top of the bars
ax = (bursts_df_sleep.groupby(["Category", "sub_ID"])["Duration"].count().unstack().T.div(bursts_df_sleep.groupby(["sub_ID"])["Duration"].count(), axis = 0)*100).plot(kind = "bar", figsize = (15, 9), stacked = True)
ax.set_xlabel("Subjects", fontsize = 19)
ax.set_ylabel("Movement percentage", fontsize = 19)
ax.set_title("Movement distribution", fontsize = 21)
ax.set_xticks([])
# put legend outside the plot
ax.legend(title = "Category", fontsize = 14, frameon = True, fancybox = True, shadow = True, loc = "upper right", bbox_to_anchor=(1.18, 1)) 
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))

Sleep vs Wake

In [93]:
perc_sleep = pd.DataFrame(bursts_df_sleep.groupby(["Category"])["Duration"].count() / bursts_df_sleep["Duration"].count())
perc_wake = pd.DataFrame(bursts_df_wake.groupby(["Category"])["Duration"].count() / bursts_df_wake["Duration"].count())

df = pd.concat([perc_sleep, perc_wake], axis = 1)
df.columns = ["Sleep", "Wake"]

df.plot(kind = "bar", figsize = (15, 9), stacked = True)

<Axes: xlabel='Category'>

In [96]:
size = 0.3

(ax1, ax2) = df.plot(kind = "pie", subplots = True, figsize = (15, 9), autopct = "%.1f%%", textprops = {"fontsize": 16})
ax1.set_title("Sleep", fontsize = 21)
ax1.set_ylabel("")
ax1.legend([])
ax2.set_title("Wake", fontsize = 21)
ax2.set_ylabel("")
ax2.legend([])
plt.tight_layout()

In [98]:
df["Sleep"].values

array([0.42756184, 0.23733804, 0.15253239, 0.13780919, 0.04475854])

In [260]:
singleLimb_df = bursts_df_sleep[bursts_df_sleep["Category"] == "Focal"].reset_index(drop=True)
perc_LW = singleLimb_df[singleLimb_df["Limbs"] == {"LW"}].shape[0] / singleLimb_df.shape[0]
perc_RW = singleLimb_df[singleLimb_df["Limbs"] == {"RW"}].shape[0] / singleLimb_df.shape[0]
perc_LL = singleLimb_df[singleLimb_df["Limbs"] == {"LL"}].shape[0] / singleLimb_df.shape[0]
perc_RL = singleLimb_df[singleLimb_df["Limbs"] == {"RL"}].shape[0] / singleLimb_df.shape[0]
perc_T = singleLimb_df[singleLimb_df["Limbs"] == {"T"}].shape[0] / singleLimb_df.shape[0]

perc_LW, perc_RW, perc_RL, perc_LL, perc_T

(0.25757575757575757,
 0.2493112947658402,
 0.23140495867768596,
 0.18732782369146006,
 0.0743801652892562)

In [261]:
from matplotlib.patches import ConnectionPatch

# do what above but with the sleep data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 9), gridspec_kw={'width_ratios': [2, 1]})
fig.subplots_adjust(wspace=0)

# pie chart parameters
overall_ratios_sleep = df["Sleep"].values
labels_sleep = df.index.to_list()
labels_sleep[4] = "\nUpper\nBody"

explode = [0.1, 0, 0, 0, 0]
# rotate so that first wedge is split by the x-axis
angle = -180 * overall_ratios_sleep[0]
wedges, *_ = ax1.pie(overall_ratios_sleep, autopct='%1.1f%%', startangle=angle,
                     labels=labels_sleep, explode=explode, textprops = {"fontsize": 16})

# bar chart parameters
singleLimb_ratios = [perc_LW, perc_RW, perc_RL, perc_LL, perc_T]
singleLimb_labels = ["LW", "RW", "RA", "LA", "T"]
bottom = 1
width = .2

# Adding from the top matches the legend.
for j, (height, label) in enumerate(reversed([*zip(singleLimb_ratios, singleLimb_labels)])):
    bottom -= height
    bc = ax2.bar(0, height, width, bottom=bottom, color='C0', label=label,
                 alpha=0.1 + 0.20 * j)
    ax2.bar_label(bc, labels=[f"{height:.0%}"], label_type='center', fontsize = 16)

ax2.set_title('Focal movements', fontsize = 16)
ax2.legend(frameon = True, fancybox = True, shadow = True, fontsize = 16)
ax2.axis('off')
ax2.set_xlim(- 2.5 * width, 2.5 * width)

# use ConnectionPatch to draw lines between the two plots
theta1, theta2 = wedges[0].theta1, wedges[0].theta2
center, r = wedges[0].center, wedges[0].r
bar_height = sum(singleLimb_ratios)

# draw top connecting line
x = r * np.cos(np.pi / 180 * theta2) + center[0]
y = r * np.sin(np.pi / 180 * theta2) + center[1]
con = ConnectionPatch(xyA=(-width / 2, bar_height), coordsA=ax2.transData,
                      xyB=(x, y), coordsB=ax1.transData)
con.set_color([0, 0, 0])
con.set_linewidth(4)
ax2.add_artist(con)

# draw bottom connecting line
x = r * np.cos(np.pi / 180 * theta1) + center[0]
y = r * np.sin(np.pi / 180 * theta1) + center[1]
con = ConnectionPatch(xyA=(-width / 2, 0), coordsA=ax2.transData,
                        xyB=(x, y), coordsB=ax1.transData)
con.set_color([0, 0, 0])
ax2.add_artist(con)
con.set_linewidth(4)

plt.suptitle("Sleep", fontsize = 21)

plt.show()

In [262]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/movement_distribution_sleep.png", dpi = 300, bbox_inches = 'tight')

In [253]:
# same during wake
singleLimb_df = bursts_df_wake[bursts_df_wake["Category"] == "Focal"].reset_index(drop=True)
perc_LW = singleLimb_df[singleLimb_df["Limbs"] == {"LW"}].shape[0] / singleLimb_df.shape[0]
perc_RW = singleLimb_df[singleLimb_df["Limbs"] == {"RW"}].shape[0] / singleLimb_df.shape[0]
perc_LL = singleLimb_df[singleLimb_df["Limbs"] == {"LL"}].shape[0] / singleLimb_df.shape[0]
perc_RL = singleLimb_df[singleLimb_df["Limbs"] == {"RL"}].shape[0] / singleLimb_df.shape[0]
perc_T = singleLimb_df[singleLimb_df["Limbs"] == {"T"}].shape[0] / singleLimb_df.shape[0]

In [258]:
# do what above but with the wake data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13, 9), gridspec_kw={'width_ratios': [2, 1]})
fig.subplots_adjust(wspace=0)

# pie chart parameters
overall_ratios_wake = df["Wake"].values
labels_wake = df.index.to_list()
labels_wake[4] = "\n\n\nUpper\nBody"

explode = [0.1, 0, 0, 0, 0]
# rotate so that first wedge is split by the x-axis
angle = -180 * overall_ratios_wake[0]
wedges, *_ = ax1.pie(overall_ratios_wake, autopct='%1.1f%%', startangle=angle,
                     labels=labels_wake, explode=explode, textprops = {"fontsize": 16})

# bar chart parameters
singleLimb_ratios = [perc_LW, perc_RW, perc_LL, perc_RL, perc_T]
singleLimb_labels = ["LW", "RW", "LA", "RA", "T"]
bottom = 1
width = .2

# Adding from the top matches the legend.
for j, (height, label) in enumerate(reversed([*zip(singleLimb_ratios, singleLimb_labels)])):
    bottom -= height
    bc = ax2.bar(0, height, width, bottom=bottom, color='C0', label=label,
                 alpha=0.1 + 0.20 * j)
    ax2.bar_label(bc, labels=[f"{height:.0%}"], label_type='center', fontsize = 16)

ax2.set_title('Focal movements', fontsize = 16)
ax2.legend(frameon = True, fancybox = True, shadow = True, fontsize = 16)
ax2.axis('off')
ax2.set_xlim(- 2.5 * width, 2.5 * width)

# use ConnectionPatch to draw lines between the two plots
theta1, theta2 = wedges[0].theta1, wedges[0].theta2
center, r = wedges[0].center, wedges[0].r
bar_height = sum(singleLimb_ratios)

# draw top connecting line
x = r * np.cos(np.pi / 180 * theta2) + center[0]
y = r * np.sin(np.pi / 180 * theta2) + center[1]
con = ConnectionPatch(xyA=(-width / 2, bar_height), coordsA=ax2.transData,
                      xyB=(x, y), coordsB=ax1.transData)
con.set_color([0, 0, 0])
con.set_linewidth(4)
ax2.add_artist(con)

# draw bottom connecting line
x = r * np.cos(np.pi / 180 * theta1) + center[0]
y = r * np.sin(np.pi / 180 * theta1) + center[1]
con = ConnectionPatch(xyA=(-width / 2, 0), coordsA=ax2.transData,
                        xyB=(x, y), coordsB=ax1.transData, linestyle = "-")
con.set_color([0, 0, 0])
ax2.add_artist(con)
con.set_linewidth(4)

plt.suptitle("Wake", fontsize = 21)

plt.show()

In [259]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/movement_distribution_wake.png", dpi = 300, bbox_inches = 'tight')

## 1st part vs 2nd of sleep

In [3]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815", "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

tst = {sub: 0 for sub in subjects}
tst_1st_half = {sub: 0 for sub in subjects}
tst_2nd_half = {sub: 0 for sub in subjects}

SIB = {sub: 0 for sub in subjects}

bursts_df_1st_half = pd.DataFrame()
bursts_df_2nd_half = pd.DataFrame()

for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_FINAL_envInterp_p2p.pkl', 'rb') as f:
        bursts = pickle.load(f)

    df_merged_intervals = characterize_bursts(bursts)

    start_sleep, end_sleep = diary_SPT[sub]
    sleep_midpoint = start_sleep + (end_sleep - start_sleep) / 2

    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    # Take df_merged_intervals between spt_start and spt_end
    df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
    SIB[sub]["sib.duration"] = SIB[sub]["sib.end.time"] - SIB[sub]["sib.onset.time"]
    SIB[sub]["sub_ID"] = sub

    tst[sub] = (SIB[sub]["sib.duration"].sum()).total_seconds() / 3600
    tst_1st_half[sub] = (SIB[sub].loc[SIB[sub]["sib.onset.time"] < sleep_midpoint, "sib.duration"].sum()).total_seconds() / 3600
    tst_2nd_half[sub] = (SIB[sub].loc[SIB[sub]["sib.onset.time"] >= sleep_midpoint, "sib.duration"].sum()).total_seconds() / 3600

    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    df_merged_intervals["sub_ID"] = sub

    start_sleep = diary_SPT[sub][0]
    end_sleep = diary_SPT[sub][1]

    df_merged_intervals = df_merged_intervals.loc[(df_merged_intervals["Start"] >= start_sleep) & (df_merged_intervals["End"] <= end_sleep)]

    df_merged_intervals_1st_half = df_merged_intervals.loc[df_merged_intervals["Start"] < sleep_midpoint].reset_index(drop=True)
    df_merged_intervals_2nd_half = df_merged_intervals.loc[df_merged_intervals["Start"] >= sleep_midpoint].reset_index(drop=True)

    bursts_df_1st_half = pd.concat([bursts_df_1st_half, df_merged_intervals_1st_half])
    bursts_df_2nd_half = pd.concat([bursts_df_2nd_half, df_merged_intervals_2nd_half])

tst_df = pd.DataFrame(tst, index = ["TST"]).T.sort_index()
tst_1st_half_df = pd.DataFrame(tst_1st_half, index = ["TST_1st_half"]).T.sort_index()
tst_2nd_half_df = pd.DataFrame(tst_2nd_half, index = ["TST_2nd_half"]).T.sort_index()

In [4]:
bursts_df_1st_half.shape[0], bursts_df_2nd_half.shape[0]

(1005, 1461)

In [5]:
bursts_df_sleep1 = bursts_df_1st_half[bursts_df_1st_half["SIB"] == 1].reset_index(drop=True)
bursts_df_sleep1["Duration"] = bursts_df_sleep1["End"] - bursts_df_sleep1["Start"]
bursts_df_sleep1["Duration"] = bursts_df_sleep1["Duration"].apply(lambda x: x.total_seconds())
bursts_df_sleep1["n_limbs"] = bursts_df_sleep1["Limbs"].apply(lambda x: len(x))
bursts_df_sleep1["Category"] = "placeholder"
bursts_df_sleep1.loc[bursts_df_sleep1["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df_sleep1.loc[(bursts_df_sleep1["Limbs"] == {"LL", "RL"}) | (bursts_df_sleep1["Limbs"] == {"LL", "T"}) | (bursts_df_sleep1["Limbs"] == {"RL", "T"}) | (bursts_df_sleep1["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df_sleep1.loc[(bursts_df_sleep1["Limbs"] == {"LW", "RW"}) | (bursts_df_sleep1["Limbs"] == {"LW", "T"}) | (bursts_df_sleep1["Limbs"] == {"RW", "T"}) | (bursts_df_sleep1["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df_sleep1.loc[(bursts_df_sleep1["n_limbs"] >= 2) & (bursts_df_sleep1["n_limbs"] != 5) & (bursts_df_sleep1["Limbs"] != {"LL", "RL"}) & (bursts_df_sleep1["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_sleep1["Limbs"] != {"LW", "RW"}) & (bursts_df_sleep1["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_sleep1["Limbs"] != {"LL", "T"}) & (bursts_df_sleep1["Limbs"] != {"RL", "T"}) & (bursts_df_sleep1["Limbs"] != {"LW", "T"}) & (bursts_df_sleep1["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df_sleep1.loc[bursts_df_sleep1["n_limbs"] == 1, "Category"] = "Focal"


bursts_df_sleep2 = bursts_df_2nd_half[bursts_df_2nd_half["SIB"] == 1].reset_index(drop=True)
bursts_df_sleep2["Duration"] = bursts_df_sleep2["End"] - bursts_df_sleep2["Start"]
bursts_df_sleep2["Duration"] = bursts_df_sleep2["Duration"].apply(lambda x: x.total_seconds())
bursts_df_sleep2["n_limbs"] = bursts_df_sleep2["Limbs"].apply(lambda x: len(x))
bursts_df_sleep2["Category"] = "placeholder"
bursts_df_sleep2.loc[bursts_df_sleep2["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df_sleep2.loc[(bursts_df_sleep2["Limbs"] == {"LL", "RL"}) | (bursts_df_sleep2["Limbs"] == {"LL", "T"}) | (bursts_df_sleep2["Limbs"] == {"RL", "T"}) | (bursts_df_sleep2["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df_sleep2.loc[(bursts_df_sleep2["Limbs"] == {"LW", "RW"}) | (bursts_df_sleep2["Limbs"] == {"LW", "T"}) | (bursts_df_sleep2["Limbs"] == {"RW", "T"}) | (bursts_df_sleep2["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df_sleep2.loc[(bursts_df_sleep2["n_limbs"] >= 2) & (bursts_df_sleep2["n_limbs"] != 5) & (bursts_df_sleep2["Limbs"] != {"LL", "RL"}) & (bursts_df_sleep2["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_sleep2["Limbs"] != {"LW", "RW"}) & (bursts_df_sleep2["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_sleep2["Limbs"] != {"LL", "T"}) & (bursts_df_sleep2["Limbs"] != {"RL", "T"}) & (bursts_df_sleep2["Limbs"] != {"LW", "T"}) & (bursts_df_sleep2["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df_sleep2.loc[bursts_df_sleep2["n_limbs"] == 1, "Category"] = "Focal"

bursts_df_sleep1.shape[0], bursts_df_sleep2.shape[0]

(705, 993)

Movement index

In [415]:
freq_1 = bursts_df_sleep1.groupby(["Category", "sub_ID"])["Duration"].count() #/ (tst_1st_half_df["TST_1st_half"])
freq_2 = bursts_df_sleep2.groupby(["Category", "sub_ID"])["Duration"].count() #/ (tst_2nd_half_df["TST_2nd_half"])
freq_1 = freq_1.unstack().T.div(tst_1st_half_df["TST_1st_half"], axis = 0).T
freq_2 = freq_2.unstack().T.div(tst_2nd_half_df["TST_2nd_half"], axis = 0).T

In [416]:
freq = pd.concat([freq_1, freq_2], axis = 1)
freq

sub_ID,098,127,158,279,547,633,815,906,914,958,...,158,279,547,633,815,906,914,958,965,971
Category,,,,,,,,,,,,,,,,,,,,,
Focal,4.587706,12.841855,7.622951,6.283784,5.650558,0.941997,7.783784,9.485030,5.855563,6.567845,...,8.888889,3.661793,20.991254,2.336145,12.031332,10.933452,6.133460,11.368421,21.847390,13.823038
Full Body,1.889055,3.995244,3.934426,1.621622,2.081784,7.849978,10.378378,3.449102,2.576448,4.652223,...,4.301075,6.408137,6.997085,9.811811,15.415144,13.506029,5.792712,2.066986,2.891566,7.212020
Lower Body,3.778111,0.285375,0.983607,1.013514,3.568773,2.197994,0.972973,0.862275,1.405335,0.820981,...,2.580645,4.577241,2.448980,6.541207,1.127937,3.537293,9.881685,1.377990,5.783133,6.310518
Mixed,2.428786,4.565993,0.245902,0.608108,1.189591,3.139991,2.270270,2.299401,1.171113,4.378563,...,1.146953,0.457724,4.198251,5.606749,2.631854,4.502010,1.362991,3.100478,5.140562,3.305509
Upper Body,1.619190,NaN,0.983607,0.202703,0.892193,0.627998,1.621622,2.874251,0.468445,2.189282,...,0.573477,0.915448,1.049563,NaN,1.503916,1.607861,NaN,1.033493,0.321285,0.300501


Duration or AUC

In [34]:
freq_1 = bursts_df_sleep1.groupby(["Category", "sub_ID"])["AUC"].mean()
freq_2 = bursts_df_sleep2.groupby(["Category", "sub_ID"])["AUC"].mean()
freq_1 = freq_1.unstack()/1000
freq_2 = freq_2.unstack()/1000

In [35]:
# merge and boxplot each movement 1st vs 2nd half
freq = pd.concat([freq_1, freq_2], axis = 1).T
# freq.columns = ["1st_half", "2nd_half"]
freq["part"] = "1st half"
freq.reset_index(drop=True, inplace = True)
freq.loc[12:, "part"] = "2nd half"
freq["sub_ID"] = sorted(subjects) + sorted(subjects)

# freq = freq.unstack().T
# freq["part"] = freq.index.get_level_values(0)
# freq["sub_ID"] = freq.index.get_level_values(1)
# freq.reset_index(drop = True, inplace = True)

freq = freq.melt(id_vars = ["part", "sub_ID"], value_vars = [ "Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"], var_name = "Category", value_name = "Frequency")

fig, ax = plt.subplots(figsize = (15, 9))
sns.boxplot(x = "Category", y = "Frequency", hue = "part", data = freq, showfliers = False, ax = ax)
sns.stripplot(x = "Category", y = "Frequency", hue = "part", data = freq, dodge = True, jitter = True, palette = "dark:black", alpha = 0.52, s = 7, ax = ax, legend = False)
ax.set_xlabel("Category", fontsize = 19)
ax.set_ylabel("Movement AUC (g•s)", fontsize = 19)
ax.legend(fontsize = 19, frameon = True, fancybox = True, shadow = True, loc = "center")
ax.set_xlabel("")
plt.xticks(rotation = 18, fontsize = 16)
plt.yticks(fontsize = 16);
# plt.ylim(0, freq["Frequency"].max() + 5)

In [37]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/movement_AUC_1st_vs_2nd_half.png", dpi = 300, bbox_inches = 'tight')

In [36]:
# Wilcoxon signed-rank test for each category: 1st vs 2nd half
from scipy.stats import wilcoxon
p_values = {}
for cat in ["Full Body", "Lower Body", "Upper Body", "Mixed", "Focal"]:
    print(cat), print(wilcoxon(freq_1.loc[cat], freq_2.loc[cat], nan_policy = 'omit'))
    p_values[cat] = wilcoxon(freq_1.loc[cat], freq_2.loc[cat], nan_policy = 'omit')[1]

# Draw p-values on top of the boxplot
y_max = freq["Frequency"].max()
y_min = freq["Frequency"].min()
y_range = y_max - y_min
y_max = y_max + 0.1 * y_range
y_min = y_min - 0.1 * y_range
y_range = y_max - y_min
y_pos = y_max - 0.2 * y_range

for i, cat in enumerate(["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]):
    ax.text(i, y_pos, f"p = {p_values[cat]:.3f}", color = "black", fontsize = 16, ha = "center")


Full Body
WilcoxonResult(statistic=35.0, pvalue=0.791015625)
Lower Body
WilcoxonResult(statistic=11.0, pvalue=0.02685546875)
Upper Body
WilcoxonResult(statistic=16.0, pvalue=0.49609375)
Mixed
WilcoxonResult(statistic=23.0, pvalue=0.2333984375)
Focal
WilcoxonResult(statistic=25.0, pvalue=0.30126953125)


# Table OP final

In [4]:
part2_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms2.out/"
part3_outputFolder = "/Volumes/Untitled/rehab/GGIR/GGIR_output_lw_TIB/output_lw_data/meta/ms3.out/"
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815", "127", "914", "965"]

SIB_GGIR = {sub: pyreadr.read_r(part3_outputFolder + "LW_" + sub + ".CWA.RData")['sib.cla.sum'][["sib.onset.time", "sib.end.time"]] for sub in subjects}

tst = {sub: 0 for sub in subjects}
twt = {sub: 0 for sub in subjects}

SIB = {sub: 0 for sub in subjects}

bursts_df = pd.DataFrame()

for i, sub in enumerate(subjects):
    SIB_GGIR[sub]["sib.onset.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.onset.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.end.time"] = pd.to_datetime(SIB_GGIR[sub]["sib.end.time"].values).tz_localize(None)
    SIB_GGIR[sub]["sib.duration"] = SIB_GGIR[sub]["sib.end.time"] - SIB_GGIR[sub]["sib.onset.time"]

    with open(f'/Volumes/Untitled/rehab/data/{sub}/bursts_FINAL_envInterp_p2p.pkl', 'rb') as f:
        bursts = pickle.load(f)

    df_merged_intervals = characterize_bursts(bursts)
    spt_start = diary_SPT[sub][0] - pd.Timedelta('10 min')
    spt_end = diary_SPT[sub][1] + pd.Timedelta('5 min')

    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)
    SIB[sub] = SIB_GGIR[sub][(SIB_GGIR[sub]["sib.onset.time"] >= spt_start) & (SIB_GGIR[sub]["sib.end.time"] <= spt_end)].reset_index(drop=True)

    # Take df_merged_intervals between spt_start and spt_end
    df_merged_intervals = df_merged_intervals[(df_merged_intervals["Start"] >= spt_start) & (df_merged_intervals["End"] <= spt_end)].reset_index(drop=True) 

    SIB[sub]["awake.duration"] = SIB[sub]["sib.onset.time"].shift(-1) - SIB[sub]["sib.end.time"]
    SIB[sub]["sib.duration"] = SIB[sub]["sib.end.time"] - SIB[sub]["sib.onset.time"]
    SIB[sub]["sub_ID"] = sub

    tst[sub] = (SIB[sub]["sib.duration"].sum()).total_seconds() / 3600
    twt[sub] = (SIB[sub]["awake.duration"].sum()).total_seconds() / 3600

    df_merged_intervals["SIB"] = 0
    for i, row in SIB[sub].iterrows():
        df_merged_intervals.loc[(df_merged_intervals["Start"] >= row["sib.onset.time"] + pd.Timedelta("5s")) & (df_merged_intervals["End"] <= row["sib.end.time"] - pd.Timedelta("5s")), "SIB"] = 1

    df_merged_intervals["sub_ID"] = sub

    start_sleep = diary_SPT[sub][0]
    end_sleep = diary_SPT[sub][1]

    df_merged_intervals = df_merged_intervals.loc[(df_merged_intervals["Start"] >= start_sleep) & (df_merged_intervals["End"] <= end_sleep)]

    bursts_df = pd.concat([bursts_df, df_merged_intervals])

In [5]:
bursts_df_sleep = bursts_df[bursts_df["SIB"] == 1].reset_index(drop=True)
bursts_df_sleep["Duration"] = bursts_df_sleep["End"] - bursts_df_sleep["Start"]
bursts_df_sleep["Duration"] = bursts_df_sleep["Duration"].apply(lambda x: x.total_seconds())
bursts_df_sleep["n_limbs"] = bursts_df_sleep["Limbs"].apply(lambda x: len(x))
bursts_df_sleep["Category"] = "placeholder"
bursts_df_sleep["Laterality"] = "Bilateral"
bursts_df_sleep.loc[bursts_df_sleep["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_df_sleep["Limbs"] == {"LL", "T"}) | (bursts_df_sleep["Limbs"] == {"RL", "T"}) | (bursts_df_sleep["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LW", "RW"}) | (bursts_df_sleep["Limbs"] == {"LW", "T"}) | (bursts_df_sleep["Limbs"] == {"RW", "T"}) | (bursts_df_sleep["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df_sleep.loc[(bursts_df_sleep["n_limbs"] >= 2) & (bursts_df_sleep["n_limbs"] != 5) & (bursts_df_sleep["Limbs"] != {"LL", "RL"}) & (bursts_df_sleep["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_sleep["Limbs"] != {"LW", "RW"}) & (bursts_df_sleep["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_sleep["Limbs"] != {"LL", "T"}) & (bursts_df_sleep["Limbs"] != {"RL", "T"}) & (bursts_df_sleep["Limbs"] != {"LW", "T"}) & (bursts_df_sleep["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df_sleep.loc[bursts_df_sleep["n_limbs"] == 1, "Category"] = "Focal"
bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "LW"}) | (bursts_df_sleep["Limbs"] == {"RL", "RW"}) | (bursts_df_sleep["Limbs"] == {"LL", "LW", "T"}) | (bursts_df_sleep["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "T"}) | (bursts_df_sleep["Limbs"] == {"LW", "T"}) | (bursts_df_sleep["Limbs"] == {"RL", "T"}) | (bursts_df_sleep["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_df_sleep.loc[(bursts_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_df_sleep["Limbs"] == {"LW"}) | (bursts_df_sleep["Limbs"] == {"RW"}) | (bursts_df_sleep["Limbs"] == {"LL"}) | (bursts_df_sleep["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_df_sleep.loc[bursts_df_sleep["Limbs"] == {"T"}, "Laterality"] = "None"

bursts_df_wake = bursts_df[bursts_df["SIB"] == 0].reset_index(drop=True)
bursts_df_wake["Duration"] = bursts_df_wake["End"] - bursts_df_wake["Start"]
bursts_df_wake["Duration"] = bursts_df_wake["Duration"].apply(lambda x: x.total_seconds())
bursts_df_wake["n_limbs"] = bursts_df_wake["Limbs"].apply(lambda x: len(x))
bursts_df_wake["Category"] = "placeholder"
bursts_df_wake["Laterality"] = "Bilateral"
bursts_df_wake.loc[bursts_df_wake["n_limbs"] == 5, "Category"] = "Full Body"
bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "RL"}) | (bursts_df_wake["Limbs"] == {"LL", "T"}) | (bursts_df_wake["Limbs"] == {"RL", "T"}) | (bursts_df_wake["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LW", "RW"}) | (bursts_df_wake["Limbs"] == {"LW", "T"}) | (bursts_df_wake["Limbs"] == {"RW", "T"}) | (bursts_df_wake["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_df_wake.loc[(bursts_df_wake["n_limbs"] >= 2) & (bursts_df_wake["n_limbs"] != 5) & (bursts_df_wake["Limbs"] != {"LL", "RL"}) & (bursts_df_wake["Limbs"] != {"LL", "RL", "T"}) & (bursts_df_wake["Limbs"] != {"LW", "RW"}) & (bursts_df_wake["Limbs"] != {"LW", "RW", "T"}) & (bursts_df_wake["Limbs"] != {"LL", "T"}) & (bursts_df_wake["Limbs"] != {"RL", "T"}) & (bursts_df_wake["Limbs"] != {"LW", "T"}) & (bursts_df_wake["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_df_wake.loc[bursts_df_wake["n_limbs"] == 1, "Category"] = "Focal"
bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "LW"}) | (bursts_df_wake["Limbs"] == {"RL", "RW"}) | (bursts_df_wake["Limbs"] == {"LL", "LW", "T"}) | (bursts_df_wake["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "T"}) | (bursts_df_wake["Limbs"] == {"LW", "T"}) | (bursts_df_wake["Limbs"] == {"RL", "T"}) | (bursts_df_wake["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_df_wake.loc[(bursts_df_wake["Limbs"] == {"LL", "RL"}) | (bursts_df_wake["Limbs"] == {"LW"}) | (bursts_df_wake["Limbs"] == {"RW"}) | (bursts_df_wake["Limbs"] == {"LL"}) | (bursts_df_wake["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_df_wake.loc[bursts_df_wake["Limbs"] == {"T"}, "Laterality"] = "None"

tst_df = pd.DataFrame(tst, index = ["TST"]).T.sort_index()
twt_df = pd.DataFrame(twt, index = ["TWT"]).T.sort_index()

In [67]:
bursts_df_sleep.head()

,Start,End,AUC,p2p,PC,transition,Limbs,SIB,sub_ID,Duration,n_limbs,Category,Laterality
0,2024-02-28 23:00:01.360290051,2024-02-28 23:00:02.460289955,4478.029974,59.446242,0.0,None,"{RL, LL, T}",1,158,1.099999,3,Lower Body,Bilateral
1,2024-02-28 23:00:30.130290031,2024-02-28 23:00:32.030289888,5565.732278,48.247713,0.0,None,{LL},1,158,1.899999,1,Focal,Unilateral
2,2024-02-28 23:00:43.494869947,2024-02-28 23:00:43.874870062,593.812924,16.318746,0.0,None,{T},1,158,0.380000,1,Focal,None
3,2024-02-28 23:02:28.765630007,2024-02-28 23:02:31.105629921,6965.555964,44.916305,0.0,None,"{LW, T}",1,158,2.339999,2,Upper Body,Unilateral
4,2024-02-28 23:03:30.215630054,2024-02-28 23:03:32.105629921,6534.732075,50.164798,0.0,None,"{LW, T}",1,158,1.889999,2,Upper Body,Unilateral


In [68]:
# create a df where, for each movement, there is the median (25th percentile - 75th percentile): for example for Full Body: 0.5 (0.3 - 0.7)
df_index = bursts_df_sleep.groupby(["Category", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(tst_df["TST"], axis = 0).describe()
df_index_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_index_sleep.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_index_sleep.columns:
    med = df_index.loc["50%", movement].round(1)
    perc25 = df_index.loc["25%", movement].round(1)
    perc75 = df_index.loc["75%", movement].round(1)
    df_index_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_index_sleep = df_index_sleep.T
df_index_sleep.columns = ["Index (n/h)"]

df_duration = bursts_df_sleep.groupby(["Category", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_duration_sleep.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_duration_sleep.columns:
    med = df_duration.loc["50%", movement].round(1)
    perc25 = df_duration.loc["25%", movement].round(1)
    perc75 = df_duration.loc["75%", movement].round(1)
    df_duration_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_duration_sleep = df_duration_sleep.T
df_duration_sleep.columns = ["Duration (s)"]

df_p2p = (bursts_df_sleep.groupby(["Category", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_p2p_sleep.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_p2p_sleep.columns:
    med = df_p2p.loc["50%", movement].round(2)
    perc25 = df_p2p.loc["25%", movement].round(2)
    perc75 = df_p2p.loc["75%", movement].round(2)
    df_p2p_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_p2p_sleep = df_p2p_sleep.T
df_p2p_sleep.columns = ["p2p (g)"]

df_AUC = (bursts_df_sleep.groupby(["Category", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_sleep = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_AUC_sleep.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_AUC_sleep.columns:
    med = df_AUC.loc["50%", movement].round(1)
    perc25 = df_AUC.loc["25%", movement].round(1)
    perc75 = df_AUC.loc["75%", movement].round(1)
    df_AUC_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_AUC_sleep = df_AUC_sleep.T
df_AUC_sleep.columns = ["AUC (g·s)"]

table_movements_sleep = pd.concat([df_index_sleep, df_duration_sleep, df_p2p_sleep, df_AUC_sleep], axis = 1)

df_index_laterality = bursts_df_sleep.groupby(["Laterality", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(tst_df["TST"], axis = 0).describe()
df_index_laterality_sleep = pd.DataFrame(["a", "a"]).T
df_index_laterality_sleep.columns = ["Unilateral", "Bilateral"]
for movement in df_index_laterality_sleep.columns:
    med = df_index_laterality.loc["50%", movement].round(1)
    perc25 = df_index_laterality.loc["25%", movement].round(1)
    perc75 = df_index_laterality.loc["75%", movement].round(1)
    df_index_laterality_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_index_laterality_sleep = df_index_laterality_sleep.T
df_index_laterality_sleep.columns = ["Index (n/h)"]
df_index_laterality_sleep

df_duration_laterality = bursts_df_sleep.groupby(["Laterality", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_laterality_sleep = pd.DataFrame(["a", "a"]).T
df_duration_laterality_sleep.columns = ["Unilateral", "Bilateral"]
for movement in df_duration_laterality_sleep.columns:
    med = df_duration_laterality.loc["50%", movement].round(1)
    perc25 = df_duration_laterality.loc["25%", movement].round(1)
    perc75 = df_duration_laterality.loc["75%", movement].round(1)
    df_duration_laterality_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_duration_laterality_sleep = df_duration_laterality_sleep.T
df_duration_laterality_sleep.columns = ["Duration (s)"]
df_duration_laterality_sleep

df_p2p_laterality = (bursts_df_sleep.groupby(["Laterality", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_laterality_sleep = pd.DataFrame(["a", "a"]).T
df_p2p_laterality_sleep.columns = ["Unilateral", "Bilateral"]
for movement in df_p2p_laterality_sleep.columns:
    med = df_p2p_laterality.loc["50%", movement].round(2)
    perc25 = df_p2p_laterality.loc["25%", movement].round(2)
    perc75 = df_p2p_laterality.loc["75%", movement].round(2)
    df_p2p_laterality_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_p2p_laterality_sleep = df_p2p_laterality_sleep.T
df_p2p_laterality_sleep.columns = ["p2p (g)"]
df_p2p_laterality_sleep

df_AUC_laterality = (bursts_df_sleep.groupby(["Laterality", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_laterality_sleep = pd.DataFrame(["a", "a"]).T
df_AUC_laterality_sleep.columns = ["Unilateral", "Bilateral"]
for movement in df_AUC_laterality_sleep.columns:
    med = df_AUC_laterality.loc["50%", movement].round(1)
    perc25 = df_AUC_laterality.loc["25%", movement].round(1)
    perc75 = df_AUC_laterality.loc["75%", movement].round(1)
    df_AUC_laterality_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_AUC_laterality_sleep = df_AUC_laterality_sleep.T
df_AUC_laterality_sleep.columns = ["AUC (g·s)"]
df_AUC_laterality_sleep

table_laterality_sleep = pd.concat([df_index_laterality_sleep, df_duration_laterality_sleep, df_p2p_laterality_sleep, df_AUC_laterality_sleep], axis = 1)

table_sleep = pd.concat([table_movements_sleep, table_laterality_sleep], axis = 0)

In [69]:
# same as above but for wake
df_index = bursts_df_wake.groupby(["Category", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(twt_df["TWT"], axis = 0).describe()
df_index_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_index_wake.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_index_wake.columns:
    med = df_index.loc["50%", movement].round(1)
    perc25 = df_index.loc["25%", movement].round(1)
    perc75 = df_index.loc["75%", movement].round(1)
    df_index_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_index_wake = df_index_wake.T
df_index_wake.columns = ["Index (n/h)"]

df_duration = bursts_df_wake.groupby(["Category", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_duration_wake.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_duration_wake.columns:
    med = df_duration.loc["50%", movement].round(1)
    perc25 = df_duration.loc["25%", movement].round(1)
    perc75 = df_duration.loc["75%", movement].round(1)
    df_duration_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_duration_wake = df_duration_wake.T
df_duration_wake.columns = ["Duration (s)"]

df_p2p = (bursts_df_wake.groupby(["Category", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_p2p_wake.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_p2p_wake.columns:
    med = df_p2p.loc["50%", movement].round(2)
    perc25 = df_p2p.loc["25%", movement].round(2)
    perc75 = df_p2p.loc["75%", movement].round(2)
    df_p2p_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_p2p_wake = df_p2p_wake.T
df_p2p_wake.columns = ["p2p (g)"]

df_AUC = (bursts_df_wake.groupby(["Category", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_wake = pd.DataFrame(["a", "a", "a", "a", "a"]).T
df_AUC_wake.columns = ["Focal", "Full Body", "Lower Body", "Upper Body", "Mixed"]
for movement in df_AUC_wake.columns:
    med = df_AUC.loc["50%", movement].round(1)
    perc25 = df_AUC.loc["25%", movement].round(1)
    perc75 = df_AUC.loc["75%", movement].round(1)
    df_AUC_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_AUC_wake = df_AUC_wake.T
df_AUC_wake.columns = ["AUC (g·s)"]

table_movements_wake = pd.concat([df_index_wake, df_duration_wake, df_p2p_wake, df_AUC_wake], axis = 1)

df_index_laterality = bursts_df_wake.groupby(["Laterality", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(tst_df["TST"], axis = 0).describe()
df_index_laterality_wake = pd.DataFrame(["a", "a"]).T
df_index_laterality_wake.columns = ["Unilateral", "Bilateral"]
for movement in df_index_laterality_wake.columns:
    med = df_index_laterality.loc["50%", movement].round(1)
    perc25 = df_index_laterality.loc["25%", movement].round(1)
    perc75 = df_index_laterality.loc["75%", movement].round(1)
    df_index_laterality_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_index_laterality_wake = df_index_laterality_wake.T
df_index_laterality_wake.columns = ["Index (n/h)"]

df_duration_laterality = bursts_df_wake.groupby(["Laterality", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_laterality_wake = pd.DataFrame(["a", "a"]).T
df_duration_laterality_wake.columns = ["Unilateral", "Bilateral"]
for movement in df_duration_laterality_wake.columns:
    med = df_duration_laterality.loc["50%", movement].round(1)
    perc25 = df_duration_laterality.loc["25%", movement].round(1)
    perc75 = df_duration_laterality.loc["75%", movement].round(1)
    df_duration_laterality_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_duration_laterality_wake = df_duration_laterality_wake.T
df_duration_laterality_wake.columns = ["Duration (s)"]

df_p2p_laterality = (bursts_df_wake.groupby(["Laterality", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_laterality_wake = pd.DataFrame(["a", "a"]).T
df_p2p_laterality_wake.columns = ["Unilateral", "Bilateral"]
for movement in df_p2p_laterality_wake.columns:
    med = df_p2p_laterality.loc["50%", movement].round(2)
    perc25 = df_p2p_laterality.loc["25%", movement].round(2)
    perc75 = df_p2p_laterality.loc["75%", movement].round(2)
    df_p2p_laterality_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_p2p_laterality_wake = df_p2p_laterality_wake.T
df_p2p_laterality_wake.columns = ["p2p (g)"]

df_AUC_laterality = (bursts_df_wake.groupby(["Laterality", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_laterality_wake = pd.DataFrame(["a", "a"]).T
df_AUC_laterality_wake.columns = ["Unilateral", "Bilateral"]
for movement in df_AUC_laterality_wake.columns:
    med = df_AUC_laterality.loc["50%", movement].round(1)
    perc25 = df_AUC_laterality.loc["25%", movement].round(1)
    perc75 = df_AUC_laterality.loc["75%", movement].round(1)
    df_AUC_laterality_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_AUC_laterality_wake = df_AUC_laterality_wake.T
df_AUC_laterality_wake.columns = ["AUC (g·s)"]

table_laterality_wake = pd.concat([df_index_laterality_wake, df_duration_laterality_wake, df_p2p_laterality_wake, df_AUC_laterality_wake], axis = 1)

table_wake = pd.concat([table_movements_wake, table_laterality_wake], axis = 0)

In [70]:
bursts_df_sleep.loc[bursts_df_sleep["Category"] != "Focal", "Category"] = "Non Focal"

df_index_focal = bursts_df_sleep.groupby(["Category", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(tst_df["TST"], axis = 0).describe()
df_index_focal_sleep = pd.DataFrame(["a", "a"]).T
df_index_focal_sleep.columns = ["Focal", "Non Focal"]
for movement in df_index_focal_sleep.columns:
    med = df_index_focal.loc["50%", movement].round(1)
    perc25 = df_index_focal.loc["25%", movement].round(1)
    perc75 = df_index_focal.loc["75%", movement].round(1)
    df_index_focal_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_index_focal_sleep = df_index_focal_sleep.T
df_index_focal_sleep.columns = ["Index (n/h)"]

df_duration_focal = bursts_df_sleep.groupby(["Category", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_focal_sleep = pd.DataFrame(["a", "a"]).T
df_duration_focal_sleep.columns = ["Focal", "Non Focal"]
for movement in df_duration_focal_sleep.columns:
    med = df_duration_focal.loc["50%", movement].round(1)
    perc25 = df_duration_focal.loc["25%", movement].round(1)
    perc75 = df_duration_focal.loc["75%", movement].round(1)
    df_duration_focal_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_duration_focal_sleep = df_duration_focal_sleep.T
df_duration_focal_sleep.columns = ["Duration (s)"]

df_p2p_focal = (bursts_df_sleep.groupby(["Category", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_focal_sleep = pd.DataFrame(["a", "a"]).T
df_p2p_focal_sleep.columns = ["Focal", "Non Focal"]
for movement in df_p2p_focal_sleep.columns:
    med = df_p2p_focal.loc["50%", movement].round(2)
    perc25 = df_p2p_focal.loc["25%", movement].round(2)
    perc75 = df_p2p_focal.loc["75%", movement].round(2)
    df_p2p_focal_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_p2p_focal_sleep = df_p2p_focal_sleep.T
df_p2p_focal_sleep.columns = ["p2p (g)"]

df_AUC_focal = (bursts_df_sleep.groupby(["Category", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_focal_sleep = pd.DataFrame(["a", "a"]).T
df_AUC_focal_sleep.columns = ["Focal", "Non Focal"]
for movement in df_AUC_focal_sleep.columns:
    med = df_AUC_focal.loc["50%", movement].round(1)
    perc25 = df_AUC_focal.loc["25%", movement].round(1)
    perc75 = df_AUC_focal.loc["75%", movement].round(1)
    df_AUC_focal_sleep[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_AUC_focal_sleep = df_AUC_focal_sleep.T
df_AUC_focal_sleep.columns = ["AUC (g·s)"]

table_focal_sleep = pd.concat([df_index_focal_sleep, df_duration_focal_sleep, df_p2p_focal_sleep, df_AUC_focal_sleep], axis = 1)

table_sleep = pd.concat([table_sleep, table_focal_sleep.loc["Non Focal"].to_frame().T], axis = 0)

bursts_df_wake.loc[bursts_df_wake["Category"] != "Focal", "Category"] = "Non Focal"

df_index_focal = bursts_df_wake.groupby(["Category", "sub_ID"])["Duration"].describe()["count"].unstack().T.div(twt_df["TWT"], axis = 0).describe()
df_index_focal_wake = pd.DataFrame(["a", "a"]).T
df_index_focal_wake.columns = ["Focal", "Non Focal"]
for movement in df_index_focal_wake.columns:
    med = df_index_focal.loc["50%", movement].round(1)
    perc25 = df_index_focal.loc["25%", movement].round(1)
    perc75 = df_index_focal.loc["75%", movement].round(1)
    df_index_focal_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_index_focal_wake = df_index_focal_wake.T
df_index_focal_wake.columns = ["Index (n/h)"]

df_duration_focal = bursts_df_wake.groupby(["Category", "sub_ID"])["Duration"].describe()["mean"].unstack().T.describe()
df_duration_focal_wake = pd.DataFrame(["a", "a"]).T
df_duration_focal_wake.columns = ["Focal", "Non Focal"]
for movement in df_duration_focal_wake.columns:
    med = df_duration_focal.loc["50%", movement].round(1)
    perc25 = df_duration_focal.loc["25%", movement].round(1)
    perc75 = df_duration_focal.loc["75%", movement].round(1)
    df_duration_focal_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_duration_focal_wake = df_duration_focal_wake.T
df_duration_focal_wake.columns = ["Duration (s)"]

df_p2p_focal = (bursts_df_wake.groupby(["Category", "sub_ID"])["p2p"].describe()["mean"].unstack().T/1000).describe()
df_p2p_focal_wake = pd.DataFrame(["a", "a"]).T
df_p2p_focal_wake.columns = ["Focal", "Non Focal"]
for movement in df_p2p_focal_wake.columns:
    med = df_p2p_focal.loc["50%", movement].round(2)
    perc25 = df_p2p_focal.loc["25%", movement].round(2)
    perc75 = df_p2p_focal.loc["75%", movement].round(2)
    df_p2p_focal_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_p2p_focal_wake = df_p2p_focal_wake.T
df_p2p_focal_wake.columns = ["p2p (g)"]

df_AUC_focal = (bursts_df_wake.groupby(["Category", "sub_ID"])["AUC"].describe()["mean"].unstack().T/1000).describe()
df_AUC_focal_wake = pd.DataFrame(["a", "a"]).T
df_AUC_focal_wake.columns = ["Focal", "Non Focal"]
for movement in df_AUC_focal_wake.columns:
    med = df_AUC_focal.loc["50%", movement].round(1)
    perc25 = df_AUC_focal.loc["25%", movement].round(1)
    perc75 = df_AUC_focal.loc["75%", movement].round(1)
    df_AUC_focal_wake[movement] = str(med) + " (" + str(perc25) + "-" + str(perc75) + ")"
df_AUC_focal_wake = df_AUC_focal_wake.T
df_AUC_focal_wake.columns = ["AUC (g·s)"]

table_focal_wake = pd.concat([df_index_focal_wake, df_duration_focal_wake, df_p2p_focal_wake, df_AUC_focal_wake], axis = 1)

table_wake = pd.concat([table_wake, table_focal_wake.loc["Non Focal"].to_frame().T], axis = 0)

In [64]:
table_wake

,Index (n/h),Duration (s),p2p (g),AUC (g·s)
Focal,12.6 (9.0-19.1),1.6 (1.3-1.9),0.04 (0.03-0.05),5.5 (3.1-7.5)
Full Body,58.9 (55.1-61.5),19.7 (16.0-30.4),3.85 (3.25-4.5),1925.9 (1567.3-3063.7)
Lower Body,6.2 (4.7-8.3),3.3 (2.3-4.9),0.09 (0.06-0.12),21.0 (8.6-31.2)
Upper Body,3.9 (1.9-5.1),5.0 (4.8-6.6),0.34 (0.22-0.46),76.6 (72.2-158.6)
Mixed,8.1 (5.3-12.7),6.0 (4.8-6.5),0.4 (0.35-0.54),81.8 (67.1-130.0)
Unilateral,1.5 (1.2-2.2),2.0 (1.4-2.2),0.05 (0.04-0.06),7.5 (4.8-11.1)
Bilateral,5.9 (5.5-9.5),17.6 (13.6-25.4),3.18 (2.69-3.76),1584.2 (1333.7-2683.8)
Non Focal,73.7 (67.5-80.9),17.0 (13.2-24.0),3.01 (2.56-3.34),1504.8 (1230.8-2333.1)
Non Focal,73.7 (67.5-80.9),17.0 (13.2-24.0),3.01 (2.56-3.34),1504.8 (1230.8-2333.1)


In [71]:
table = pd.concat([table_sleep, table_wake], axis = 1)
table

,Index (n/h),Duration (s),p2p (g),AUC (g·s),Index (n/h),Duration (s),p2p (g),AUC (g·s)
Focal,9.1 (7.6-10.7),1.3 (1.2-1.6),0.04 (0.03-0.05),4.7 (3.0-5.9),12.6 (9.0-19.1),1.6 (1.3-1.9),0.04 (0.03-0.05),5.5 (3.1-7.5)
Full Body,4.2 (3.4-5.4),9.5 (8.0-10.9),1.06 (0.95-1.25),337.6 (283.7-504.9),58.9 (55.1-61.5),19.7 (16.0-30.4),3.85 (3.25-4.5),1925.9 (1567.3-3063.7)
Lower Body,2.8 (2.0-4.3),3.1 (2.8-3.5),0.1 (0.08-0.13),16.7 (13.0-23.4),6.2 (4.7-8.3),3.3 (2.3-4.9),0.09 (0.06-0.12),21.0 (8.6-31.2)
Upper Body,0.8 (0.4-1.6),3.4 (2.8-4.5),0.12 (0.08-0.25),20.2 (13.6-46.0),3.9 (1.9-5.1),5.0 (4.8-6.6),0.34 (0.22-0.46),76.6 (72.2-158.6)
Mixed,3.0 (2.1-4.0),4.9 (4.2-5.7),0.27 (0.21-0.3),53.6 (39.1-71.2),8.1 (5.3-12.7),6.0 (4.8-6.5),0.4 (0.35-0.54),81.8 (67.1-130.0)
Unilateral,11.1 (9.4-13.6),1.8 (1.6-2.0),0.05 (0.04-0.06),7.3 (4.9-10.3),1.5 (1.2-2.2),2.0 (1.4-2.2),0.05 (0.04-0.06),7.5 (4.8-11.1)
Bilateral,8.8 (6.6-12.2),7.1 (6.2-8.0),0.67 (0.56-0.85),200.0 (162.6-251.8),5.9 (5.5-9.5),17.6 (13.6-25.4),3.18 (2.69-3.76),1584.2 (1333.7-2683.8)
Non Focal,12.0 (10.2-15.4),6.2 (5.3-6.9),0.54 (0.42-0.66),165.3 (119.2-212.3),73.7 (67.5-80.9),17.0 (13.2-24.0),3.01 (2.56-3.34),1504.8 (1230.8-2333.1)


In [72]:
table.to_excel("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/tables/table1_new_new.xlsx")

# HR response revisited

## Focal and Non Focal Sleep vs Wake

In [3]:
import matplotlib.ticker as mtick

In [29]:
subjects = ["158", "098", "633", "279", "906", "547", "971", "958", "815"]
bursts_HR = pd.read_pickle("/Volumes/Untitled/rehab/data/bursts_HR_ACC_final.pkl")
bursts_HR["ACC_response"] = bursts_HR["ACC_response"].apply(lambda x: np.array(x))
print(bursts_HR.shape)

(834, 15)


In [30]:
bursts_HR_df_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)
bursts_HR_df_sleep["Duration"] = bursts_HR_df_sleep["End"] - bursts_HR_df_sleep["Start"]
bursts_HR_df_sleep["Duration"] = bursts_HR_df_sleep["Duration"].apply(lambda x: x.total_seconds())
bursts_HR_df_sleep["n_limbs"] = bursts_HR_df_sleep["Limbs"].apply(lambda x: len(x))
bursts_HR_df_sleep["Category"] = "placeholder"
bursts_HR_df_sleep.loc[bursts_HR_df_sleep["n_limbs"] == 1, "Category"] = "Focal"
bursts_HR_df_sleep.loc[bursts_HR_df_sleep["Category"] != "Focal", "Category"] = "Non Focal"


bursts_HR_df_wake = bursts_HR[bursts_HR["SIB"] == 0].reset_index(drop=True)
bursts_HR_df_wake["Duration"] = bursts_HR_df_wake["End"] - bursts_HR_df_wake["Start"]
bursts_HR_df_wake["Duration"] = bursts_HR_df_wake["Duration"].apply(lambda x: x.total_seconds())
bursts_HR_df_wake["n_limbs"] = bursts_HR_df_wake["Limbs"].apply(lambda x: len(x))
bursts_HR_df_wake["Category"] = "placeholder"
bursts_HR_df_wake.loc[bursts_HR_df_wake["n_limbs"] == 1, "Category"] = "Focal"
bursts_HR_df_wake.loc[bursts_HR_df_wake["Category"] != "Focal", "Category"] = "Non Focal"

In [8]:
HR_response_sleep_focal = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Focal"]
HR_response_sleep_focal_sem = HR_response_sleep_focal.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_focal.dropna().shape[0])

HR_response_sleep_non_focal = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Non Focal"]
HR_response_sleep_non_focal_sem = HR_response_sleep_non_focal.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_non_focal.dropna().shape[0])

HR_response_wake_focal = bursts_HR_df_wake.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Focal"]
HR_response_wake_focal_sem = HR_response_wake_focal.dropna().to_numpy().std() / np.sqrt(HR_response_wake_focal.dropna().shape[0])

HR_response_wake_non_focal = bursts_HR_df_wake.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Non Focal"]
HR_response_wake_non_focal_sem = HR_response_wake_non_focal.dropna().to_numpy().std() / np.sqrt(HR_response_wake_non_focal.dropna().shape[0])

In [70]:
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6))

t = np.arange(-19, 50, 1)

ax1.errorbar(t, HR_response_wake_non_focal.mean(), yerr = HR_response_wake_non_focal_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")
ax1.errorbar(t, HR_response_sleep_non_focal.mean(), yerr = HR_response_sleep_non_focal_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Sleep")


ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
# ax.annotate('Movement onset', xy=(0, plt.ylim()[1]-6), xytext=(-60, plt.ylim()[1]-66),
#              textcoords='offset points', ha='right', va='bottom', fontsize=19,
#              bbox=dict(boxstyle='round,pad=0.5', fc='gray', alpha=0.2, edgecolor='black'),
#              arrowprops=dict(facecolor='black', shrink=0.05, width=2))

# plt.xlim(-15, 30)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax1.set_xlabel('Time (seconds)', fontsize = 21)
ax1.set_ylabel('HR response (%)', fontsize = 21)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax1.set_label('')
ax1.xaxis.set_tick_params(labelsize=20)
ax1.yaxis.set_tick_params(labelsize=20)
ax1.legend(frameon = True, fancybox = True, shadow = True, fontsize = 21)

ax2.errorbar(t, HR_response_wake_focal.mean(), yerr = HR_response_wake_focal_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Wake")
ax2.errorbar(t, HR_response_sleep_focal.mean(), yerr = HR_response_sleep_focal_sem, fmt = '-o', capsize=3, elinewidth=1.5, label = "Sleep")

ax2.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)

plt.xlim(-10, 20)

# plt.xticks(ticks = np.arange(-20, 40, 5), labels=np.arange(-20, 40, 5), fontsize=16)
ax2.set_xlabel('Time (seconds)', fontsize = 21)
ax2.set_ylabel('HR response (%)', fontsize = 21)
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax2.set_label('')
ax2.xaxis.set_tick_params(labelsize=20)
ax2.yaxis.set_tick_params(labelsize=20)
ax2.legend(frameon = True, fancybox = True, shadow = True, fontsize = 21)


### Numbers

Peak

In [56]:
burst_non_focal_sleep = bursts_HR_df_sleep[bursts_HR_df_sleep["Category"] == "Focal"]
burst_non_focal_sleep = burst_non_focal_sleep.groupby(["sub_ID"])["HR_response_normalized"].mean()

burst_non_focal_wake = bursts_HR_df_wake[bursts_HR_df_wake["Category"] == "Focal"]
burst_non_focal_wake = burst_non_focal_wake.groupby(["sub_ID"])["HR_response_normalized"].mean()

In [57]:
burst_non_focal_sleep

sub_ID
098    [-1.8540953787841041, -1.3546543322471596, -0....
158    [-0.286614272215478, 0.20289274727899875, -0.3...
279    [-1.4791132626841312, -2.974134703707418, -1.8...
547    [1.4304552613665562, 0.6423388122128357, -0.28...
633    [0.00968442446457137, -0.9121184984440163, -1....
815    [1.5936744195753498, 1.8116139415971724, 0.534...
906    [0.4156369192667914, 0.6955631086561689, 1.325...
958    [3.4423239658526645, 1.3853792607011723, 0.434...
971    [-0.2765750678579345, -0.37266896954880163, 0....
Name: HR_response_normalized, dtype: object

In [58]:
p_nonfocal_sleep = []
p_nonfocal_wake = []
for sub in subjects:
    p_nonfocal_sleep.append(burst_non_focal_sleep[sub].max())
    # try:
    #     p_nonfocal_wake.append(burst_non_focal_wake[sub].max())
    # except:
    #     p_nonfocal_wake.append(np.nan)

pd.Series(p_nonfocal_sleep).describe()

count     9.000000
mean      5.955708
std       2.858518
min       2.054361
25%       4.623182
50%       5.368991
75%       7.509661
max      11.998675
dtype: float64

In [40]:
from functions.plot_utils import stripplot_with_lines
peaks_sleep_vs_wake = pd.DataFrame({"Sleep": p_nonfocal_sleep, "Wake": p_nonfocal_wake})

f, ax = plt.subplots(figsize=(9, 6))
sns.boxplot(peaks_sleep_vs_wake, palette = "Set2", fill = False, ax = ax, linewidth = 2.1, width = 0.68)
jitter = 0.01
stripplot_with_lines(peaks_sleep_vs_wake, jitter, ax)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
plt.xticks(fontsize = 21)
plt.yticks(fontsize = 21)
plt.ylabel("HR peak (%)", fontsize = 21)
plt.title("HR peak associated with non-focal movements", fontsize = 24)

peaks_sleep = np.array(p_nonfocal_sleep)
peaks_wake = np.array(p_nonfocal_wake)

from scipy import stats

w_stat, p_value = stats.wilcoxon(peaks_sleep, peaks_wake)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

W-statistic: 0.0, p-value: 0.00390625


In [41]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/boxplot_peaks_sleep_wake_non_focal.png", dpi = 300, bbox_inches = 'tight')

Duration

In [49]:
d_sleep = []
d_wake = []

for sub in subjects:
    d_sleep.append(np.where(burst_non_focal_sleep[sub][20:] <= 0)[0][0])
    d_wake.append(np.where(burst_non_focal_wake[sub][20:] <= 0)[0][0])

pd.Series(d_wake).describe()

count     9.000000
mean     30.333333
std       5.431390
min      23.000000
25%      26.000000
50%      31.000000
75%      34.000000
max      39.000000
dtype: float64

In [47]:
from functions.plot_utils import stripplot_with_lines
duration_sleep_vs_wake = pd.DataFrame({"Sleep": d_sleep, "Wake": d_wake})

f, ax = plt.subplots(figsize=(9, 6))
sns.boxplot(duration_sleep_vs_wake, palette = "Set2", fill = False, ax = ax, linewidth = 2.1, width = 0.68)
jitter = 0.01
stripplot_with_lines(duration_sleep_vs_wake, jitter, ax)
plt.xticks(fontsize = 21)
plt.yticks(fontsize = 21)
plt.ylabel("HR duration (s)", fontsize = 21)
plt.title("HR response duration associated with non-focal movements", fontsize = 24)

duration_sleep = np.array(d_sleep)
duration_wake = np.array(d_wake)

from scipy import stats

w_stat, p_value = stats.wilcoxon(duration_sleep, duration_wake)

print(f"W-statistic: {w_stat}, p-value: {p_value}")

W-statistic: 0.0, p-value: 0.00390625


In [48]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/HR/boxplot_duration_sleep_wake_non_focal.png", dpi = 300, bbox_inches = 'tight')

## Non-focal (Lower-body, Upper-body, Mixed)

In [5]:
bursts_HR_df_sleep = bursts_HR[bursts_HR["SIB"] == 1].reset_index(drop=True)
bursts_HR_df_sleep["Duration"] = bursts_HR_df_sleep["End"] - bursts_HR_df_sleep["Start"]
bursts_HR_df_sleep["Duration"] = bursts_HR_df_sleep["Duration"].apply(lambda x: x.total_seconds())
bursts_HR_df_sleep["n_limbs"] = bursts_HR_df_sleep["Limbs"].apply(lambda x: len(x))
bursts_HR_df_sleep["Category"] = "placeholder"
bursts_HR_df_sleep["Laterality"] = "Bilateral"
bursts_HR_df_sleep.loc[bursts_HR_df_sleep["n_limbs"] == 5, "Category"] = "Full Body"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_HR_df_sleep["Limbs"] == {"LL", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"RL", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["Limbs"] == {"LW", "RW"}) | (bursts_HR_df_sleep["Limbs"] == {"LW", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"RW", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["n_limbs"] >= 2) & (bursts_HR_df_sleep["n_limbs"] != 5) & (bursts_HR_df_sleep["Limbs"] != {"LL", "RL"}) & (bursts_HR_df_sleep["Limbs"] != {"LL", "RL", "T"}) & (bursts_HR_df_sleep["Limbs"] != {"LW", "RW"}) & (bursts_HR_df_sleep["Limbs"] != {"LW", "RW", "T"}) & (bursts_HR_df_sleep["Limbs"] != {"LL", "T"}) & (bursts_HR_df_sleep["Limbs"] != {"RL", "T"}) & (bursts_HR_df_sleep["Limbs"] != {"LW", "T"}) & (bursts_HR_df_sleep["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_HR_df_sleep.loc[bursts_HR_df_sleep["n_limbs"] == 1, "Category"] = "Focal"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["Limbs"] == {"LL", "LW"}) | (bursts_HR_df_sleep["Limbs"] == {"RL", "RW"}) | (bursts_HR_df_sleep["Limbs"] == {"LL", "LW", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["Limbs"] == {"LL", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"LW", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"RL", "T"}) | (bursts_HR_df_sleep["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR_df_sleep.loc[(bursts_HR_df_sleep["Limbs"] == {"LL", "RL"}) | (bursts_HR_df_sleep["Limbs"] == {"LW"}) | (bursts_HR_df_sleep["Limbs"] == {"RW"}) | (bursts_HR_df_sleep["Limbs"] == {"LL"}) | (bursts_HR_df_sleep["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_HR_df_sleep.loc[bursts_HR_df_sleep["Limbs"] == {"T"}, "Laterality"] = "None"

bursts_HR_df_wake = bursts_HR[bursts_HR["SIB"] == 0].reset_index(drop=True)
bursts_HR_df_wake["Duration"] = bursts_HR_df_wake["End"] - bursts_HR_df_wake["Start"]
bursts_HR_df_wake["Duration"] = bursts_HR_df_wake["Duration"].apply(lambda x: x.total_seconds())
bursts_HR_df_wake["n_limbs"] = bursts_HR_df_wake["Limbs"].apply(lambda x: len(x))
bursts_HR_df_wake["Category"] = "placeholder"
bursts_HR_df_wake["Laterality"] = "Bilateral"
bursts_HR_df_wake.loc[bursts_HR_df_wake["n_limbs"] == 5, "Category"] = "Full Body"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["Limbs"] == {"LL", "RL"}) | (bursts_HR_df_wake["Limbs"] == {"LL", "T"}) | (bursts_HR_df_wake["Limbs"] == {"RL", "T"}) | (bursts_HR_df_wake["Limbs"] == {"LL", "RL", "T"}), "Category"] = "Lower Body"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["Limbs"] == {"LW", "RW"}) | (bursts_HR_df_wake["Limbs"] == {"LW", "T"}) | (bursts_HR_df_wake["Limbs"] == {"RW", "T"}) | (bursts_HR_df_wake["Limbs"] == {"LW", "RW", "T"}), "Category"] = "Upper Body"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["n_limbs"] >= 2) & (bursts_HR_df_wake["n_limbs"] != 5) & (bursts_HR_df_wake["Limbs"] != {"LL", "RL"}) & (bursts_HR_df_wake["Limbs"] != {"LL", "RL", "T"}) & (bursts_HR_df_wake["Limbs"] != {"LW", "RW"}) & (bursts_HR_df_wake["Limbs"] != {"LW", "RW", "T"}) & (bursts_HR_df_wake["Limbs"] != {"LL", "T"}) & (bursts_HR_df_wake["Limbs"] != {"RL", "T"}) & (bursts_HR_df_wake["Limbs"] != {"LW", "T"}) & (bursts_HR_df_wake["Limbs"] != {"RW", "T"}), "Category"] = "Mixed"
bursts_HR_df_wake.loc[bursts_HR_df_wake["n_limbs"] == 1, "Category"] = "Focal"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["Limbs"] == {"LL", "LW"}) | (bursts_HR_df_wake["Limbs"] == {"RL", "RW"}) | (bursts_HR_df_wake["Limbs"] == {"LL", "LW", "T"}) | (bursts_HR_df_wake["Limbs"] == {"RL", "RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["Limbs"] == {"LL", "T"}) | (bursts_HR_df_wake["Limbs"] == {"LW", "T"}) | (bursts_HR_df_wake["Limbs"] == {"RL", "T"}) | (bursts_HR_df_wake["Limbs"] == {"RW", "T"}), "Laterality"] = "Unilateral"
bursts_HR_df_wake.loc[(bursts_HR_df_wake["Limbs"] == {"LL", "RL"}) | (bursts_HR_df_wake["Limbs"] == {"LW"}) | (bursts_HR_df_wake["Limbs"] == {"RW"}) | (bursts_HR_df_wake["Limbs"] == {"LL"}) | (bursts_HR_df_wake["Limbs"] == {"RL"}), "Laterality"] = "Unilateral"
bursts_HR_df_wake.loc[bursts_HR_df_wake["Limbs"] == {"T"}, "Laterality"] = "None"


In [42]:
HR_response_sleep_lower = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Lower Body"]
HR_response_sleep_lower_sem = HR_response_sleep_lower.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_lower.dropna().shape[0])

HR_response_sleep_upper = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Upper Body"]
HR_response_sleep_upper_sem = HR_response_sleep_upper.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_upper.dropna().shape[0])

HR_response_sleep_mixed = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Mixed"]
HR_response_sleep_mixed_sem = HR_response_sleep_mixed.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_mixed.dropna().shape[0])

In [59]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21, 6), sharey=True)

t = np.arange(-19, 50, 1)
f.subplots_adjust(wspace=0.05)

ax1.errorbar(t, HR_response_sleep_lower.mean(), yerr = HR_response_sleep_lower_sem, fmt = '-o', color = "k", capsize=3, elinewidth=1.5, label = "Lower Body")
ax2.errorbar(t, HR_response_sleep_upper.mean(), yerr = HR_response_sleep_upper_sem, fmt = '-o', color = "k", capsize=3, elinewidth=1.5, label = "Upper Body")
ax3.errorbar(t, HR_response_sleep_mixed.mean(), yerr = HR_response_sleep_mixed_sem, fmt = '-o', color = "k", capsize=3, elinewidth=1.5, label = "Mixed")

ax1.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax2.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax3.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)

ax1.set_xlim(-11, 28)
ax2.set_xlim(-11, 28)
ax3.set_xlim(-11, 28)

ax1.set_title("Lower Body Movement", fontsize = 19)
ax2.set_title("Upper Body Movement", fontsize = 19)
ax3.set_title("Mixed Movement", fontsize = 19)

ax1.set_xlabel('Time (seconds)', fontsize = 19)
ax2.set_xlabel('Time (seconds)', fontsize = 19)
ax3.set_xlabel('Time (seconds)', fontsize = 19)

ax1.set_ylabel('HR change', fontsize = 21)
ax1.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))

ax1.xaxis.set_tick_params(labelsize=19)
ax1.yaxis.set_tick_params(labelsize=19)
ax2.xaxis.set_tick_params(labelsize=19)
ax3.xaxis.set_tick_params(labelsize=19)

In [60]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/HR/HR_response_sleep_lower_upper_mixed.png", dpi = 300)

## Focal

In [77]:
HR_response_sleep_focal = bursts_HR_df_sleep.groupby(["sub_ID","Category"])["HR_response_normalized"].mean().unstack()["Focal"]
HR_response_sleep_focal_sem = HR_response_sleep_focal.dropna().to_numpy().std() / np.sqrt(HR_response_sleep_focal.dropna().shape[0])

In [85]:
f, ax= plt.subplots(1, 1, figsize=(12, 5))

t = np.arange(-19, 50, 1)

ax.errorbar(t+1, HR_response_sleep_focal.mean(), yerr = HR_response_sleep_focal_sem, fmt = '-o', color = "k", capsize=3, elinewidth=1.5, label = "Focal")

ax.axvline(x=0, color='grey', linestyle='--', linewidth=1.2)
ax.axhline(y=0, color='grey', linestyle='--', linewidth=1.2)

ax.set_xlim(-15+1, 18+1)

plt.xlabel('Time (seconds)', fontsize = 19)
plt.ylabel('HR change', fontsize = 21)
ax.yaxis.set_major_formatter(mtick.PercentFormatter(decimals=0))
ax.xaxis.set_tick_params(labelsize=19)
ax.yaxis.set_tick_params(labelsize=19)

plt.yticks(fontsize=16)
plt.title("Focal Movement", fontsize = 21)

Text(0.5, 1.0, 'Focal Movement')

In [86]:
plt.savefig("/Users/marcellosicbaldi/Library/CloudStorage/OneDrive-AlmaMaterStudiorumUniversitàdiBologna/Marcello/sleep-movement-hr/figures/HR/HR_response_sleep_focal.png", dpi = 300)